In [1]:
import csv
import pandas as pd
from pyblock2.driver.core import DMRGDriver, SymmetryTypes, MPOAlgorithmTypes
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
from pathlib import Path
import os
import time

In [2]:
def current_file_path():
    current_file = os.path.abspath(os.path.dirname(os.getcwd()))
    return current_file

# Data Storing Scripts

In [3]:
def generate_variable_name(params, **kwargs):
    """
    Generate a variable name dynamically based on the given parameters.
    Parameters:
    - params (dict): A dictionary containing 't' and 'theta_foldername' as keys.
    Returns:
    - str: The dynamically generated variable name.
    """
    t_value = params['t']
    u_value = params['u']
    if t_value != 0:
        t_formatted = "{:.1e}".format(t_value).replace("+", "").replace("e0", "e")
    else:
        t_formatted = "0e0"

    if u_value != 0:
        U_formatted = "{:.1e}".format(u_value).replace("+", "").replace("e0", "e")
    else:
        U_formatted = "0e0"
    theta_formatted = params['theta_foldername']
    return f"data_t{t_formatted}_U{U_formatted}_theta{theta_formatted}"

## HDF5 Files

In [4]:
def generate_unique_key(params, **kwargs):
    """
    Generate a unique key for the HDF5 dataset based on the provided parameters.

    Parameters:
    - params (dict): Dictionary containing 't' and 'u' values.
    - kwargs (dict): Dictionary of additional key-value pairs for key generation.

    Returns:
    - str: The formatted unique key.
    """
    t_value = params['t']
    u_value = params['u']

    if t_value != 0:
        t_formatted = "{:.1e}".format(t_value).replace("+", "").replace(".", "_").replace("e0", "e").replace("-", "")
    else:
        t_formatted = "0e0"

    if u_value != 0:
        U_formatted = "{:.1e}".format(u_value).replace("+", "").replace(".", "_").replace("e0", "e").replace("-", "")
    else:
        U_formatted = "0e0"

    if 'std_deviation_numberOp' in kwargs:
        key = f"std_deviation_numberOp_U{U_formatted}_t{t_formatted}"
    elif 'exp_nOp' in kwargs:
        key = f"expnOp_U{U_formatted}_t{t_formatted}"
    elif 'gndstate_energy' in kwargs:
        key = f"gndenergy_U{U_formatted}_t{t_formatted}"
    elif 'firstexcitedenergy' in kwargs:
        key = f"firstexcitedenergy_U{U_formatted}_t{t_formatted}"
    return key


In [5]:
def generating_complete_path_hdf5(filename, **kwargs):
    current_file = current_file_path()
    if params['theta_foldername'] == '0':
        if 'Correlation' in kwargs:
            base_dir = Path.cwd() / 'AHM_Data_Codes' / f"theta{params['theta_foldername']}" / f"L{params['L']}" / 'Correl_matrix'
        elif 'entropy' in kwargs:
            base_dir = Path.cwd() / 'AHM_Data_Codes' / f"theta{params['theta_foldername']}" / f"L{params['L']}" / 'Entropy'
        elif 'expnop' in kwargs:
            base_dir = Path.cwd() / 'AHM_Data_Codes' / f"theta{params['theta_foldername']}" / f"L{params['L']}" / 'Exp_nOp'
        else:      
            base_dir = Path.cwd() / 'AHM_Data_Codes' / f"theta{params['theta_foldername']}" / f"L{params['L']}" / 'Remainig_Data'
    else:
        if 'Correlation' in kwargs:
            base_dir = Path.cwd() / 'AHM_Data_Codes' / f"theta_{params['theta_foldername']}" / f"L{params['L']}" / 'Correl_matrix'
        elif 'entropy' in kwargs:
            base_dir = Path.cwd() / 'AHM_Data_Codes' / f"theta_{params['theta_foldername']}" / f"L{params['L']}" / 'Entropy'
        elif 'expnop' in kwargs:
            base_dir = Path.cwd() / 'AHM_Data_Codes' / f"theta_{params['theta_foldername']}" / f"L{params['L']}" / 'Exp_nOp'
        else:
            base_dir = Path.cwd() / 'AHM_Data_Codes' / f"theta_{params['theta_foldername']}" / f"L{params['L']}" / 'Remainig_Data'
    base_dir.mkdir(parents=True, exist_ok=True)
    # Define the full file path
    file_path = base_dir / filename
    return file_path

In [6]:
def create_folders_and_store_data(params, data, filename, **kwargs):
    """
    Create folders dynamically based on theta and L values, and store data in a file.
    Writes data - overwrite if file already exists
    Parameters:
    - theta (float or str): The value of theta (e.g., 0, pi/4, pi/2, etc.).
    - L (int): The integer value of L.
    - data (DataFrame): The data to be stored.
    - filename (str): The name of the file to store data in.
    """
    # Define the base directory
    if params['theta_foldername'] == '0':
        if 'Correlation' in kwargs:
            base_dir = Path.cwd() / 'AHM_Data_Codes' / f"theta{params['theta_foldername']}" / f"L{params['L']}" / 'Correl_matrix'
        elif 'entropy' in kwargs:
            base_dir = Path.cwd() / 'AHM_Data_Codes' / f"theta{params['theta_foldername']}" / f"L{params['L']}" / 'Entropy'
        elif 'expnop' in kwargs:
            base_dir = Path.cwd() / 'AHM_Data_Codes' / f"theta{params['theta_foldername']}" / f"L{params['L']}" / 'Exp_nOp'
        else:      
            base_dir = Path.cwd() / 'AHM_Data_Codes' / f"theta{params['theta_foldername']}" / f"L{params['L']}" / 'Remainig_Data'
    else:
        if 'Correlation' in kwargs:
            base_dir = Path.cwd() / 'AHM_Data_Codes' / f"theta_{params['theta_foldername']}" / f"L{params['L']}" / 'Correl_matrix'
        elif 'entropy' in kwargs:
            base_dir = Path.cwd() / 'AHM_Data_Codes' / f"theta_{params['theta_foldername']}" / f"L{params['L']}" / 'Entropy'
        elif 'expnop' in kwargs:
            base_dir = Path.cwd() / 'AHM_Data_Codes' / f"theta_{params['theta_foldername']}" / f"L{params['L']}" / 'Exp_nOp'
        else:
            base_dir = Path.cwd() / 'AHM_Data_Codes' / f"theta_{params['theta_foldername']}" / f"L{params['L']}" / 'Remainig_Data'
    # Create directories if they don't exist
    base_dir.mkdir(parents=True, exist_ok=True)
    # Define the full file path
    file_path = base_dir / filename
    # Writing the data to a HDF5 file in the created directory
    data.to_hdf(file_path, key=kwargs['key'], mode='w')

# filename = 'testing.csv'
# data=  pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
# key = 'testing'
# params['L'] = 10
# params['theta_foldername'] = '0'
# create_folders_and_store_data(params, data, filename, key = key, Entropy = True)

In [7]:
def write_data_to_hdf5(params, data, file_name, **kwargs):
    """
    This is useful only when the folder exists and you want to create new hdf5 file.
    Write a single dataset to an HDF5 file under a specified key.
    Parameters:
    - file_name (str): The name of the HDF5 file.
    - data (pd.DataFrame or pd.Series): The data to write to the HDF5 file.
    - key (str): The key under which the data will be stored in the HDF5 file.
    """
    data.to_hdf(file_name, key=kwargs['key'], mode='a', format='table')

In [8]:
def generate_file_name_hdf5(params, **kwargs):
    """
    Generate a file name dynamically based on the given parameters.
    Parameters:
    - params (dict): A dictionary containing 'L', 'nmax', and 't' as keys.
    Returns:
    - str: The dynamically generated file name.
    """
    # Format the 't' value to scientific notation if needed
    t_value = params['t']
    u_value = params['u']
    
    if t_value != 0:
        t_formatted = "{:.1e}".format(t_value).replace("+", "").replace("e0", "e")
    else:
        t_formatted = "0e0"

    if u_value != 0:
        U_formatted = "{:.1e}".format(u_value).replace("+", "").replace("e0", "e")
    else:
        U_formatted = "0e0"

    if 'Correlation' in kwargs:
        file_name = f"L{params['L']}_t{t_formatted}_Nmax{params['NB_MAX']}_Nbosons{params['N_BOSON']}_U{U_formatted}_CorrelationMatrix.h5"
    elif 'entropy' in kwargs:
        file_name = f"L{params['L']}_t{t_formatted}_Nmax{params['NB_MAX']}_Nbosons{params['N_BOSON']}_U{U_formatted}_entropy.h5"
    elif 'expnop' in kwargs:
        file_name = f"L{params['L']}_t{t_formatted}_Nmax{params['NB_MAX']}_Nbosons{params['N_BOSON']}_U{U_formatted}_expnop.h5"
    else:
        file_name = f"L{params['L']}_t{t_formatted}_Nmax{params['NB_MAX']}_Nbosons{params['N_BOSON']}_U{U_formatted}.h5"
    return file_name


In [9]:
def load_matrix_from_hdf5(filepath, **kwargs):
    """
    Load a matrix from an HDF5 file using pandas.
    Parameters:
    - filepath (str or Path): The path to the HDF5 file.
    - key (str): The key to the dataset to load (default is 'correlation_matrix').
    Returns:
    - ndarray: The matrix loaded from the HDF5 file as a NumPy array.
    """
    df = pd.read_hdf(filepath, key=kwargs['key'])
    return df.values  # Convert the DataFrame to a NumPy array


In [10]:
def append_data_to_hdf5(params, data, filename, **kwargs):
    """
    Create folders dynamically based on theta and L values, and append data to an HDF5 file.
    
    If the file does not exist, it will be created. If it does exist, the data will be appended.

    Parameters:
    - theta (float or str): The value of theta (e.g., 0, pi/4, pi/2, etc.).
    - L (int): The integer value of L.
    - data (DataFrame or Series): The data to append.
    - filename (str): The name of the HDF5 file to store/append data.
    - key (str): The key under which the data is stored in the HDF5 file.
    """
    # Define the base directory
    if params['theta_foldername'] == '0':
        if 'Correlation' in kwargs:
            base_dir = Path.cwd() / 'AHM_Data_Codes' / f"theta{params['theta_foldername']}" / f"L{params['L']}" / 'Correl_matrix'
        elif 'entropy' in kwargs:
            base_dir = Path.cwd() / 'AHM_Data_Codes' / f"theta{params['theta_foldername']}" / f"L{params['L']}" / 'Entropy'
        elif 'expnop' in kwargs:
            base_dir = Path.cwd() / 'AHM_Data_Codes' / f"theta{params['theta_foldername']}" / f"L{params['L']}" / 'Exp_nOp'
        else:      
            base_dir = Path.cwd() / 'AHM_Data_Codes' / f"theta{params['theta_foldername']}" / f"L{params['L']}" / 'Remainig_Data'
    else:
        if 'Correlation' in kwargs:
            base_dir = Path.cwd() / 'AHM_Data_Codes' / f"theta_{params['theta_foldername']}" / f"L{params['L']}" / 'Correl_matrix'
        elif 'entropy' in kwargs:
            base_dir = Path.cwd() / 'AHM_Data_Codes' / f"theta_{params['theta_foldername']}" / f"L{params['L']}" / 'Entropy'
        elif 'expnop' in kwargs:
            base_dir = Path.cwd() / 'AHM_Data_Codes' / f"theta_{params['theta_foldername']}" / f"L{params['L']}" / 'Exp_nOp'
        else:
            base_dir = Path.cwd() / 'AHM_Data_Codes' / f"theta_{params['theta_foldername']}" / f"L{params['L']}" / 'Remainig_Data'
    
    base_dir.mkdir(parents=True, exist_ok=True)
    file_path = base_dir / filename
    # Append the data to the HDF5 file (create the file if it doesn't exist)
    data.to_hdf(file_path, key=kwargs['key'], mode='a', index=False, append=True)

# Data for a $\theta$, $L$, $N\_{Boson}$, $N\_{Max}$, $U$

In [13]:
params={}
params['L'], params['N_BOSON'] = 40, 40
params['theta_foldername_list'] = ['0', 'Piby4', 'Piby2', '3Piby4', 'Pi']
params['theta'], params['theta_foldername'] = np.pi, params['theta_foldername_list'][4]
params['t'] = 1
params['u'] = 1
params['mu'] = 0
params['NB_MAX'] = 3 # max n_boson per site
params['theta_list'] = [0, np.pi/4, 3*np.pi/4, np.pi/2, np.pi]
params['t_list'] = np.arange(0.01, 2, 0.1)

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SAny|SymmetryTypes.CPX, n_threads=4)

driver.set_symmetry_groups("U1")
Q = driver.bw.SX

In [14]:
site_basis, site_ops = [], []
for k in range(params['L']):
    basis = [(Q(i), 1) for i in range(params['NB_MAX'] + 1)] 
    ops = {
        "": np.identity(params['NB_MAX'] + 1),                           # identity
        "C": np.diag(np.sqrt(np.arange(1, params['NB_MAX'] + 1)), k=-1), # b+
        "D": np.diag(np.sqrt(np.arange(1, params['NB_MAX'] + 1)), k=1),  # b
        "N": np.diag(np.arange(0, params['NB_MAX'] + 1), k=0),           # particle number
        "A": np.diag(np.sqrt(np.arange(1, params['NB_MAX'] + 1))*np.exp(1j * params['theta'] * np.arange(params['NB_MAX'])), k=-1), # A+_withPhase  
        "B": np.diag(np.sqrt(np.arange(1, params['NB_MAX'] + 1))*np.exp(-1j * params['theta'] * np.arange(params['NB_MAX'])), k=1), # A_withPhase  
    }
    site_basis.append(basis)
    site_ops.append(ops)

## First Excited Gap, Correlation, Number Operator and Entropy Data

#### $U = 0$

In [20]:
start_time = time.time()
#########################################

params['u'] = 0
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=60, bond_dims=[50] * 4 + [100] * 4 + [150] * 4 + [200] * 4 + [250] * 4 + [300] * 4 + [350] * 4 + [400] * 4 + [450] * 4 + [500] * 4 + [550] * 4 + [600] * 4 + [650] * 4 + [700] * 4,
    noises=[1e-4] * 30 + [1e-5] * 29 + [0], thrds=[1e-10] * 60, dav_max_iter=30, iprint=1)

#########################################
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")


Sweep =    0 | Direction =  forward | Bond dimension =   50 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.464 | E[  2] =     -46.1685991489    -45.9957023194 | DW = 3.46402e-05

Sweep =    1 | Direction = backward | Bond dimension =   50 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      2.342 | E[  2] =     -47.9846711845    -47.8661407465 | DE = -1.87e+00 | DW = 8.37581e-05

Sweep =    2 | Direction =  forward | Bond dimension =   50 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      5.102 | E[  2] =     -48.5845653584    -48.5295329688 | DE = -6.63e-01 | DW = 4.68341e-05

Sweep =    3 | Direction = backward | Bond dimension =   50 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      7.160 | E[  2] =     -49.3390908592    -49.3200920480 | DE = -7.91e-01 | DW = 1.35182e-04

Sweep =    4 | Direction =  forward | Bond dimension =  100 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =     12.628 | E[  

In [16]:
kets = [driver.split_mps(mps, ir, tag="KET-%d" % ir) for ir in range(mps.nroots)]

##### First Excitation Gap

In [17]:
gndenergy = pd.DataFrame({'gndenergy':[np.power(np.abs(energy[1]-energy[0]),2)]})
filename = generate_file_name_hdf5(params)
## Complete path
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, gndstate_energy = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, gndenergy, filename1, key = key)

In [18]:
## Testing
gndenergy = load_matrix_from_hdf5(filename1, key = key)
print(gndenergy[0][0])

0.002624015771146903


In [ ]:
0.00023694153658134542 --> 600, ket -- 50, sweeps 10
0.00019492616866190876 
0.0003254248894209413 --> 650, ket -- 50, sweeps 10

##### Entropy

In [329]:
entropy1 = driver.get_bipartite_entanglement(kets[0])

###### HDF5 Files

In [330]:
entropy = pd.DataFrame(entropy1)
filename = generate_file_name_hdf5(params, entropy = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, entropy = True)
append_data_to_hdf5(params, entropy, filename, key = 'entropy', entropy = True)

##### Correlation Matrix

In [331]:
hop_exp_val = np.zeros((params['L'], params['L']), dtype=complex)
for i in range(params['L']):
    for j in range(i, params['L']):
        b = driver.expr_builder()
        b.add_term("CD", np.array([i, j]), 1)  # Hopping operator between i and j            
        hop_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
        hop_exp_val[i, j] = driver.expectation(kets[0], hop_mpo, kets[0])
        hop_exp_val[j, i] = np.conjugate(hop_exp_val[i, j])

In [332]:
correl_matrix = pd.DataFrame(hop_exp_val)
filename = generate_file_name_hdf5(params, Correlation = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, Correlation = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

##### NumberOperator and Its Standard Deviation

In [333]:
std_val, std_nOp = 0, 0
_exp_nop = []
for i in range(params['L']):
    b = driver.expr_builder()
    b.add_term("N", np.array([i]), 1)
## Calculate <N>   
    nOp_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    impo = driver.get_identity_mpo()

    nOp_exp = driver.expectation(kets[0], nOp_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0])
    _exp_nop.append(nOp_exp)
## Calculate <N^2>
    b = driver.expr_builder()
    b.add_term("NN", np.repeat([i], 2), 1)
    
    NN_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    NN_exp = driver.expectation(kets[0], NN_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0]) 
## Calculate Standard Deviation
    std_val+=np.sqrt(NN_exp - (nOp_exp*nOp_exp))        
std_nOp = (std_val/params['L'])

###### Expectation value of Number Operator on each site

In [334]:
density_nop = pd.DataFrame(_exp_nop)
filename = generate_file_name_hdf5(params, expnop = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, expnop = True)
key = generate_unique_key(params, exp_nOp = True)
append_data_to_hdf5(params, density_nop, filename, key = key, expnop = True)

In [335]:
# key = generate_unique_key(params, exp_nOp = True)
# density_nop = load_matrix_from_hdf5(filename1, key = key)
# print(density_nop)

###### Standard Deviation

In [336]:
std_deviation_numberOp = pd.DataFrame({'std_nOp':[std_nOp]})
filename = generate_file_name_hdf5(params)
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, std_deviation_numberOp = True)

## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, std_deviation_numberOp, filename1, key = key)

####### Use this only for the first time when taking data for a theta #######
## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, std_deviation_numberOp, filename, key = key)

In [337]:
key = generate_unique_key(params, std_deviation_numberOp = True)
std_nop = load_matrix_from_hdf5(filename1, key = key)
print(std_nop[0][0])

(0.6289591764258072+3.323139823956348e-20j)


In [338]:
with pd.HDFStore(filename1, 'r') as hdf:
    # List all keys in the file
    keys = hdf.keys()
    print("Keys in HDF5 file:")
    for key in keys:
        print(key)

Keys in HDF5 file:
/gndenergy_U0e0_t1_0e0
/std_deviation_numberOp_U0e0_t1_0e0


In [ ]:
kets = [driver.split_mps(mps, ir, tag="KET-%d" % ir) for ir in range(mps.nroots)]

In [ ]:
entropy1 = driver.get_bipartite_entanglement(kets[0])

In [ ]:
params['t'] = 0
update_csv_column("theta0_AHMDmrg_data.csv", 'Entropy', 't', params['t'], str(entropy1))

In [ ]:
print(kets[0])
print(entropy1)

In [ ]:
entropy2 = driver.get_bipartite_entanglement(kets[1])
print(entropy2)

In [ ]:
df = pd.read_csv("theta0_AHMDmrg_data.csv")
params['t'] = 0
condition = df['t'] == params['t']
df.loc[condition, 'Entropy'] = str(entropy1) * condition.sum()

# Step 4: Save the updated DataFrame back to the CSV file
df.to_csv("theta0_AHMDmrg_data.csv", index=False)


In [ ]:
df.to_pickle("dataframe.pkl")

In [ ]:
def convert_stringlist_to_float_list(string):
    # Remove the square brackets and split by spaces
    float_list = list(map(float, string.strip('[]').split()))
    return float_list

def get_data_by_conditions(df, conditions, columns):
    condition = pd.Series([True] * len(df)) 
    for key, value in conditions.items():
        condition &= df[key] == value
    filtered_df = df[condition]
    return filtered_df[columns] if not filtered_df.empty else None
colors1 = plt.cm.Reds(np.linspace(0.5, 0.95, len(params['theta_list'])))
conditions = {'t': 0} 
keywords = ['Entropy']
result = get_data_by_conditions(df, conditions, keywords)
entropy = convert_stringlist_to_float_list(result.values[0][0])
print(entropy)
plt.plot(list(range(len(entropy))), entropy, "--o", label=f'$θ$={params['theta_list'][0]:.2f}', color = colors1[0], ms=8)
plt.legend(bbox_to_anchor=(1.04, 0.5), loc="center left", borderaxespad=0)
plt.xlabel('Different Combinations of Partition of two subsystems')
plt.ylabel('Entropy')
U, t = params['u'], 0
plt.title(f'Entanglement Scaling Laws for J = {t}, U={U}')
plt.grid(True)
plt.show() 

#### $U = 0.1$

In [42]:
start_time = time.time()
#########################################

params['u'] = 0.1
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4 + [200] * 4 + [250] * 4 + [300] * 4 + [350] * 4 + [400] * 4 + [450] * 4 + [500] * 4 + [550] * 4 + [600] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)

#########################################
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

Execution time: 149.20 seconds


In [43]:
kets = [driver.split_mps(mps, ir, tag="KET-%d" % ir) for ir in range(mps.nroots)]

##### First Excitation Gap

In [44]:
gndenergy = pd.DataFrame({'gndenergy':[np.power(np.abs(energy[1]-energy[0]),2)]})
filename = generate_file_name_hdf5(params)
## Complete path
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, gndstate_energy = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, gndenergy, filename1, key = key)

In [45]:
## Testing
gndenergy = load_matrix_from_hdf5(filename1, key = key)
print(gndenergy[0][0])

0.0015714134280301892


##### Entropy

In [46]:
entropy1 = driver.get_bipartite_entanglement(kets[0])

###### HDF5 Files

In [47]:
entropy = pd.DataFrame(entropy1)
filename = generate_file_name_hdf5(params, entropy = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, entropy = True)
append_data_to_hdf5(params, entropy, filename1, key = 'entropy', entropy = True)

##### Correlation Matrix

In [48]:
hop_exp_val = np.zeros((params['L'], params['L']), dtype=complex)
for i in range(params['L']):
    for j in range(i, params['L']):
        b = driver.expr_builder()
        b.add_term("CD", np.array([i, j]), 1)  # Hopping operator between i and j            
        hop_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
        hop_exp_val[i, j] = driver.expectation(kets[0], hop_mpo, kets[0])
        hop_exp_val[j, i] = np.conjugate(hop_exp_val[i, j])

In [49]:
correl_matrix = pd.DataFrame(hop_exp_val)
filename = generate_file_name_hdf5(params, Correlation = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, Correlation = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

##### NumberOperator and Its Standard Deviation

In [50]:
std_val, std_nOp = 0, 0
_exp_nop = []
for i in range(params['L']):
    b = driver.expr_builder()
    b.add_term("N", np.array([i]), 1)
## Calculate <N>   
    nOp_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    impo = driver.get_identity_mpo()

    nOp_exp = driver.expectation(kets[0], nOp_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0])
    _exp_nop.append(nOp_exp)
## Calculate <N^2>
    b = driver.expr_builder()
    b.add_term("NN", np.repeat([i], 2), 1)
    
    NN_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    NN_exp = driver.expectation(kets[0], NN_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0]) 
## Calculate Standard Deviation
    std_val+=np.sqrt(NN_exp - (nOp_exp*nOp_exp))        
std_nOp = (std_val/params['L'])

###### Expectation value of Number Operator on each site

In [51]:
density_nop = pd.DataFrame(_exp_nop)
filename = generate_file_name_hdf5(params, expnop = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, expnop = True)
key = generate_unique_key(params, exp_nOp = True)
append_data_to_hdf5(params, density_nop, filename1, key = key, expnop = True)

In [52]:
# key = generate_unique_key(params, exp_nOp = True)
# density_nop = load_matrix_from_hdf5(filename1, key = key)
# print(density_nop)

###### Standard Deviation

In [53]:
std_deviation_numberOp = pd.DataFrame({'std_nOp':[std_nOp]})
filename = generate_file_name_hdf5(params)
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, std_deviation_numberOp = True)

## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, std_deviation_numberOp, filename1, key = key)

####### Use this only for the first time when taking data for a theta #######
## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, std_deviation_numberOp, filename, key = key)

In [54]:
key = generate_unique_key(params, std_deviation_numberOp = True)
std_nop = load_matrix_from_hdf5(filename1, key = key)
print(std_nop[0][0])

(0.794926377208388-8.7020146339946e-19j)


In [55]:
with pd.HDFStore(filename1, 'r') as hdf:
    # List all keys in the file
    keys = hdf.keys()
    print("Keys in HDF5 file:")
    for key in keys:
        print(key)

Keys in HDF5 file:
/gndenergy_U1_0e01_t1_0e0
/std_deviation_numberOp_U1_0e01_t1_0e0


#### $U = 0.2$

In [56]:
start_time = time.time()
#########################################

params['u'] = 0.2
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4 + [200] * 4 + [250] * 4 + [300] * 4 + [350] * 4 + [400] * 4 + [450] * 4 + [500] * 4 + [550] * 4 + [600] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)

#########################################
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

Execution time: 142.58 seconds


In [57]:
kets = [driver.split_mps(mps, ir, tag="KET-%d" % ir) for ir in range(mps.nroots)]

##### First Excitation Gap

In [58]:
gndenergy = pd.DataFrame({'gndenergy':[np.power(np.abs(energy[1]-energy[0]),2)]})
filename = generate_file_name_hdf5(params)
## Complete path
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, gndstate_energy = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, gndenergy, filename1, key = key)

In [59]:
## Testing
gndenergy = load_matrix_from_hdf5(filename1, key = key)
print(gndenergy[0][0])

0.0010880418521860727


##### Entropy

In [60]:
entropy1 = driver.get_bipartite_entanglement(kets[0])

###### HDF5 Files

In [61]:
entropy = pd.DataFrame(entropy1)
filename = generate_file_name_hdf5(params, entropy = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, entropy = True)
append_data_to_hdf5(params, entropy, filename, key = 'entropy', entropy = True)

##### Correlation Matrix

In [62]:
hop_exp_val = np.zeros((params['L'], params['L']), dtype=complex)
for i in range(params['L']):
    for j in range(i, params['L']):
        b = driver.expr_builder()
        b.add_term("CD", np.array([i, j]), 1)  # Hopping operator between i and j            
        hop_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
        hop_exp_val[i, j] = driver.expectation(kets[0], hop_mpo, kets[0])
        hop_exp_val[j, i] = np.conjugate(hop_exp_val[i, j])

In [63]:
correl_matrix = pd.DataFrame(hop_exp_val)
filename = generate_file_name_hdf5(params, Correlation = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, Correlation = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

##### NumberOperator and Its Standard Deviation

In [64]:
std_val, std_nOp = 0, 0
_exp_nop = []
for i in range(params['L']):
    b = driver.expr_builder()
    b.add_term("N", np.array([i]), 1)
## Calculate <N>   
    nOp_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    impo = driver.get_identity_mpo()

    nOp_exp = driver.expectation(kets[0], nOp_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0])
    _exp_nop.append(nOp_exp)
## Calculate <N^2>
    b = driver.expr_builder()
    b.add_term("NN", np.repeat([i], 2), 1)
    
    NN_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    NN_exp = driver.expectation(kets[0], NN_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0]) 
## Calculate Standard Deviation
    std_val+=np.sqrt(NN_exp - (nOp_exp*nOp_exp))        
std_nOp = (std_val/params['L'])

###### Expectation value of Number Operator on each site

In [65]:
density_nop = pd.DataFrame(_exp_nop)
filename = generate_file_name_hdf5(params, expnop = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, expnop = True)
key = generate_unique_key(params, exp_nOp = True)
append_data_to_hdf5(params, density_nop, filename, key = key, expnop = True)

In [66]:
# key = generate_unique_key(params, exp_nOp = True)
# density_nop = load_matrix_from_hdf5(filename1, key = key)
# print(density_nop)

###### Standard Deviation

In [67]:
std_deviation_numberOp = pd.DataFrame({'std_nOp':[std_nOp]})
filename = generate_file_name_hdf5(params)
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, std_deviation_numberOp = True)

## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, std_deviation_numberOp, filename1, key = key)

####### Use this only for the first time when taking data for a theta #######
## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, std_deviation_numberOp, filename, key = key)

In [68]:
key = generate_unique_key(params, std_deviation_numberOp = True)
std_nop = load_matrix_from_hdf5(filename1, key = key)
print(std_nop[0][0])

(0.7635076892736257+3.1416591148420717e-18j)


In [69]:
with pd.HDFStore(filename1, 'r') as hdf:
    # List all keys in the file
    keys = hdf.keys()
    print("Keys in HDF5 file:")
    for key in keys:
        print(key)

Keys in HDF5 file:
/gndenergy_U2_0e01_t1_0e0
/std_deviation_numberOp_U2_0e01_t1_0e0


#### $U = 0.3$

In [70]:
start_time = time.time()
#########################################

params['u'] = 0.3
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4 + [200] * 4 + [250] * 4 + [300] * 4 + [350] * 4 + [400] * 4 + [450] * 4 + [500] * 4 + [550] * 4 + [600] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)

#########################################
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

Execution time: 143.02 seconds


In [71]:
kets = [driver.split_mps(mps, ir, tag="KET-%d" % ir) for ir in range(mps.nroots)]

##### First Excitation Gap

In [72]:
gndenergy = pd.DataFrame({'gndenergy':[np.power(np.abs(energy[1]-energy[0]),2)]})
filename = generate_file_name_hdf5(params)
## Complete path
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, gndstate_energy = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, gndenergy, filename1, key = key)

In [73]:
## Testing
gndenergy = load_matrix_from_hdf5(filename1, key = key)
print(gndenergy[0][0])

0.0011539117091653305


##### Entropy

In [74]:
entropy1 = driver.get_bipartite_entanglement(kets[0])

###### HDF5 Files

In [75]:
entropy = pd.DataFrame(entropy1)
filename = generate_file_name_hdf5(params, entropy = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, entropy = True)
append_data_to_hdf5(params, entropy, filename, key = 'entropy', entropy = True)

##### Correlation Matrix

In [76]:
hop_exp_val = np.zeros((params['L'], params['L']), dtype=complex)
for i in range(params['L']):
    for j in range(i, params['L']):
        b = driver.expr_builder()
        b.add_term("CD", np.array([i, j]), 1)  # Hopping operator between i and j            
        hop_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
        hop_exp_val[i, j] = driver.expectation(kets[0], hop_mpo, kets[0])
        hop_exp_val[j, i] = np.conjugate(hop_exp_val[i, j])

In [77]:
correl_matrix = pd.DataFrame(hop_exp_val)
filename = generate_file_name_hdf5(params, Correlation = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, Correlation = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

##### NumberOperator and Its Standard Deviation

In [78]:
std_val, std_nOp = 0, 0
_exp_nop = []
for i in range(params['L']):
    b = driver.expr_builder()
    b.add_term("N", np.array([i]), 1)
## Calculate <N>   
    nOp_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    impo = driver.get_identity_mpo()

    nOp_exp = driver.expectation(kets[0], nOp_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0])
    _exp_nop.append(nOp_exp)
## Calculate <N^2>
    b = driver.expr_builder()
    b.add_term("NN", np.repeat([i], 2), 1)
    
    NN_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    NN_exp = driver.expectation(kets[0], NN_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0]) 
## Calculate Standard Deviation
    std_val+=np.sqrt(NN_exp - (nOp_exp*nOp_exp))        
std_nOp = (std_val/params['L'])

###### Expectation value of Number Operator on each site

In [79]:
density_nop = pd.DataFrame(_exp_nop)
filename = generate_file_name_hdf5(params, expnop = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, expnop = True)
key = generate_unique_key(params, exp_nOp = True)
append_data_to_hdf5(params, density_nop, filename, key = key, expnop = True)

In [80]:
# key = generate_unique_key(params, exp_nOp = True)
# density_nop = load_matrix_from_hdf5(filename1, key = key)
# print(density_nop)

###### Standard Deviation

In [81]:
std_deviation_numberOp = pd.DataFrame({'std_nOp':[std_nOp]})
filename = generate_file_name_hdf5(params)
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, std_deviation_numberOp = True)

## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, std_deviation_numberOp, filename1, key = key)

####### Use this only for the first time when taking data for a theta #######
## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, std_deviation_numberOp, filename, key = key)

In [82]:
key = generate_unique_key(params, std_deviation_numberOp = True)
std_nop = load_matrix_from_hdf5(filename1, key = key)
print(std_nop[0][0])

(0.7039209107450427-7.547488197791026e-21j)


In [83]:
with pd.HDFStore(filename1, 'r') as hdf:
    # List all keys in the file
    keys = hdf.keys()
    print("Keys in HDF5 file:")
    for key in keys:
        print(key)

Keys in HDF5 file:
/gndenergy_U3_0e01_t1_0e0
/std_deviation_numberOp_U3_0e01_t1_0e0


#### $U = 0.4$

In [84]:
start_time = time.time()
#########################################

params['u'] = 0.4
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4 + [200] * 4 + [250] * 4 + [300] * 4 + [350] * 4 + [400] * 4 + [450] * 4 + [500] * 4 + [550] * 4 + [600] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)

#########################################
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

Execution time: 149.25 seconds


In [85]:
kets = [driver.split_mps(mps, ir, tag="KET-%d" % ir) for ir in range(mps.nroots)]

##### First Excitation Gap

In [86]:
gndenergy = pd.DataFrame({'gndenergy':[np.power(np.abs(energy[1]-energy[0]),2)]})
filename = generate_file_name_hdf5(params)
## Complete path
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, gndstate_energy = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, gndenergy, filename1, key = key)

In [87]:
## Testing
gndenergy = load_matrix_from_hdf5(filename1, key = key)
print(gndenergy[0][0])

0.0015630026808742465


##### Entropy

In [88]:
entropy1 = driver.get_bipartite_entanglement(kets[0])

###### HDF5 Files

In [89]:
entropy = pd.DataFrame(entropy1)
filename = generate_file_name_hdf5(params, entropy = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, entropy = True)
append_data_to_hdf5(params, entropy, filename, key = 'entropy', entropy = True)

##### Correlation Matrix

In [90]:
hop_exp_val = np.zeros((params['L'], params['L']), dtype=complex)
for i in range(params['L']):
    for j in range(i, params['L']):
        b = driver.expr_builder()
        b.add_term("CD", np.array([i, j]), 1)  # Hopping operator between i and j            
        hop_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
        hop_exp_val[i, j] = driver.expectation(kets[0], hop_mpo, kets[0])
        hop_exp_val[j, i] = np.conjugate(hop_exp_val[i, j])

In [91]:
correl_matrix = pd.DataFrame(hop_exp_val)
filename = generate_file_name_hdf5(params, Correlation = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, Correlation = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

##### NumberOperator and Its Standard Deviation

In [92]:
std_val, std_nOp = 0, 0
_exp_nop = []
for i in range(params['L']):
    b = driver.expr_builder()
    b.add_term("N", np.array([i]), 1)
## Calculate <N>   
    nOp_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    impo = driver.get_identity_mpo()

    nOp_exp = driver.expectation(kets[0], nOp_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0])
    _exp_nop.append(nOp_exp)
## Calculate <N^2>
    b = driver.expr_builder()
    b.add_term("NN", np.repeat([i], 2), 1)
    
    NN_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    NN_exp = driver.expectation(kets[0], NN_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0]) 
## Calculate Standard Deviation
    std_val+=np.sqrt(NN_exp - (nOp_exp*nOp_exp))        
std_nOp = (std_val/params['L'])

###### Expectation value of Number Operator on each site

In [93]:
density_nop = pd.DataFrame(_exp_nop)
filename = generate_file_name_hdf5(params, expnop = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, expnop = True)
key = generate_unique_key(params, exp_nOp = True)
append_data_to_hdf5(params, density_nop, filename, key = key, expnop = True)

In [94]:
# key = generate_unique_key(params, exp_nOp = True)
# density_nop = load_matrix_from_hdf5(filename1, key = key)
# print(density_nop)

###### Standard Deviation

In [95]:
std_deviation_numberOp = pd.DataFrame({'std_nOp':[std_nOp]})
filename = generate_file_name_hdf5(params)
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, std_deviation_numberOp = True)

## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, std_deviation_numberOp, filename1, key = key)

####### Use this only for the first time when taking data for a theta #######
## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, std_deviation_numberOp, filename, key = key)

In [96]:
key = generate_unique_key(params, std_deviation_numberOp = True)
std_nop = load_matrix_from_hdf5(filename1, key = key)
print(std_nop[0][0])

(0.6707181475698424+6.269397721475861e-19j)


In [97]:
with pd.HDFStore(filename1, 'r') as hdf:
    # List all keys in the file
    keys = hdf.keys()
    print("Keys in HDF5 file:")
    for key in keys:
        print(key)

Keys in HDF5 file:
/gndenergy_U4_0e01_t1_0e0
/std_deviation_numberOp_U4_0e01_t1_0e0


#### $U = 0.5$

In [98]:
start_time = time.time()
#########################################

params['u'] = 0.5
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4 + [200] * 4 + [250] * 4 + [300] * 4 + [350] * 4 + [400] * 4 + [450] * 4 + [500] * 4 + [550] * 4 + [600] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)

#########################################
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

Execution time: 137.21 seconds


In [99]:
kets = [driver.split_mps(mps, ir, tag="KET-%d" % ir) for ir in range(mps.nroots)]

##### First Excitation Gap

In [100]:
gndenergy = pd.DataFrame({'gndenergy':[np.power(np.abs(energy[1]-energy[0]),2)]})
filename = generate_file_name_hdf5(params)
## Complete path
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, gndstate_energy = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, gndenergy, filename1, key = key)

In [101]:
## Testing
gndenergy = load_matrix_from_hdf5(filename1, key = key)
print(gndenergy[0][0])

0.0021403881464154458


##### Entropy

In [102]:
entropy1 = driver.get_bipartite_entanglement(kets[0])

###### HDF5 Files

In [103]:
entropy = pd.DataFrame(entropy1)
filename = generate_file_name_hdf5(params, entropy = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, entropy = True)
append_data_to_hdf5(params, entropy, filename, key = 'entropy', entropy = True)

##### Correlation Matrix

In [104]:
hop_exp_val = np.zeros((params['L'], params['L']), dtype=complex)
for i in range(params['L']):
    for j in range(i, params['L']):
        b = driver.expr_builder()
        b.add_term("CD", np.array([i, j]), 1)  # Hopping operator between i and j            
        hop_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
        hop_exp_val[i, j] = driver.expectation(kets[0], hop_mpo, kets[0])
        hop_exp_val[j, i] = np.conjugate(hop_exp_val[i, j])

In [105]:
correl_matrix = pd.DataFrame(hop_exp_val)
filename = generate_file_name_hdf5(params, Correlation = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, Correlation = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

##### NumberOperator and Its Standard Deviation

In [106]:
std_val, std_nOp = 0, 0
_exp_nop = []
for i in range(params['L']):
    b = driver.expr_builder()
    b.add_term("N", np.array([i]), 1)
## Calculate <N>   
    nOp_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    impo = driver.get_identity_mpo()

    nOp_exp = driver.expectation(kets[0], nOp_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0])
    _exp_nop.append(nOp_exp)
## Calculate <N^2>
    b = driver.expr_builder()
    b.add_term("NN", np.repeat([i], 2), 1)
    
    NN_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    NN_exp = driver.expectation(kets[0], NN_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0]) 
## Calculate Standard Deviation
    std_val+=np.sqrt(NN_exp - (nOp_exp*nOp_exp))        
std_nOp = (std_val/params['L'])

###### Expectation value of Number Operator on each site

In [107]:
density_nop = pd.DataFrame(_exp_nop)
filename = generate_file_name_hdf5(params, expnop = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, expnop = True)
key = generate_unique_key(params, exp_nOp = True)
append_data_to_hdf5(params, density_nop, filename, key = key, expnop = True)

In [108]:
# key = generate_unique_key(params, exp_nOp = True)
# density_nop = load_matrix_from_hdf5(filename1, key = key)
# print(density_nop)

###### Standard Deviation

In [109]:
std_deviation_numberOp = pd.DataFrame({'std_nOp':[std_nOp]})
filename = generate_file_name_hdf5(params)
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, std_deviation_numberOp = True)

## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, std_deviation_numberOp, filename1, key = key)

####### Use this only for the first time when taking data for a theta #######
## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, std_deviation_numberOp, filename, key = key)

In [110]:
key = generate_unique_key(params, std_deviation_numberOp = True)
std_nop = load_matrix_from_hdf5(filename1, key = key)
print(std_nop[0][0])

(0.658768254967465-5.932890704418073e-18j)


In [111]:
with pd.HDFStore(filename1, 'r') as hdf:
    # List all keys in the file
    keys = hdf.keys()
    print("Keys in HDF5 file:")
    for key in keys:
        print(key)

Keys in HDF5 file:
/gndenergy_U5_0e01_t1_0e0
/std_deviation_numberOp_U5_0e01_t1_0e0


#### $U = 0.6$

In [112]:
start_time = time.time()
#########################################

params['u'] = 0.6
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4 + [200] * 4 + [250] * 4 + [300] * 4 + [350] * 4 + [400] * 4 + [450] * 4 + [500] * 4 + [550] * 4 + [600] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)

#########################################
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

Execution time: 150.40 seconds


In [113]:
kets = [driver.split_mps(mps, ir, tag="KET-%d" % ir) for ir in range(mps.nroots)]

##### First Excitation Gap

In [114]:
gndenergy = pd.DataFrame({'gndenergy':[np.power(np.abs(energy[1]-energy[0]),2)]})
filename = generate_file_name_hdf5(params)
## Complete path
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, gndstate_energy = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, gndenergy, filename1, key = key)

In [115]:
## Testing
gndenergy = load_matrix_from_hdf5(filename1, key = key)
print(gndenergy[0][0])

0.0004685894556920996


##### Entropy

In [116]:
entropy1 = driver.get_bipartite_entanglement(kets[0])

###### HDF5 Files

In [117]:
entropy = pd.DataFrame(entropy1)
filename = generate_file_name_hdf5(params, entropy = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, entropy = True)
append_data_to_hdf5(params, entropy, filename, key = 'entropy', entropy = True)

##### Correlation Matrix

In [118]:
hop_exp_val = np.zeros((params['L'], params['L']), dtype=complex)
for i in range(params['L']):
    for j in range(i, params['L']):
        b = driver.expr_builder()
        b.add_term("CD", np.array([i, j]), 1)  # Hopping operator between i and j            
        hop_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
        hop_exp_val[i, j] = driver.expectation(kets[0], hop_mpo, kets[0])
        hop_exp_val[j, i] = np.conjugate(hop_exp_val[i, j])

In [119]:
correl_matrix = pd.DataFrame(hop_exp_val)
filename = generate_file_name_hdf5(params, Correlation = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, Correlation = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

##### NumberOperator and Its Standard Deviation

In [120]:
std_val, std_nOp = 0, 0
_exp_nop = []
for i in range(params['L']):
    b = driver.expr_builder()
    b.add_term("N", np.array([i]), 1)
## Calculate <N>   
    nOp_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    impo = driver.get_identity_mpo()

    nOp_exp = driver.expectation(kets[0], nOp_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0])
    _exp_nop.append(nOp_exp)
## Calculate <N^2>
    b = driver.expr_builder()
    b.add_term("NN", np.repeat([i], 2), 1)
    
    NN_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    NN_exp = driver.expectation(kets[0], NN_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0]) 
## Calculate Standard Deviation
    std_val+=np.sqrt(NN_exp - (nOp_exp*nOp_exp))        
std_nOp = (std_val/params['L'])

###### Expectation value of Number Operator on each site

In [121]:
density_nop = pd.DataFrame(_exp_nop)
filename = generate_file_name_hdf5(params, expnop = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, expnop = True)
key = generate_unique_key(params, exp_nOp = True)
append_data_to_hdf5(params, density_nop, filename, key = key, expnop = True)

In [122]:
# key = generate_unique_key(params, exp_nOp = True)
# density_nop = load_matrix_from_hdf5(filename1, key = key)
# print(density_nop)

###### Standard Deviation

In [123]:
std_deviation_numberOp = pd.DataFrame({'std_nOp':[std_nOp]})
filename = generate_file_name_hdf5(params)
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, std_deviation_numberOp = True)

## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, std_deviation_numberOp, filename1, key = key)

####### Use this only for the first time when taking data for a theta #######
## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, std_deviation_numberOp, filename, key = key)

In [124]:
key = generate_unique_key(params, std_deviation_numberOp = True)
std_nop = load_matrix_from_hdf5(filename1, key = key)
print(std_nop[0][0])

(0.659217576605545+7.415447086929104e-18j)


In [125]:
with pd.HDFStore(filename1, 'r') as hdf:
    # List all keys in the file
    keys = hdf.keys()
    print("Keys in HDF5 file:")
    for key in keys:
        print(key)

Keys in HDF5 file:
/gndenergy_U6_0e01_t1_0e0
/std_deviation_numberOp_U6_0e01_t1_0e0


#### $U = 0.7$

In [126]:
start_time = time.time()
#########################################

params['u'] = 0.7
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4 + [200] * 4 + [250] * 4 + [300] * 4 + [350] * 4 + [400] * 4 + [450] * 4 + [500] * 4 + [550] * 4 + [600] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)

#########################################
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

Execution time: 147.89 seconds


In [127]:
kets = [driver.split_mps(mps, ir, tag="KET-%d" % ir) for ir in range(mps.nroots)]

##### First Excitation Gap

In [128]:
gndenergy = pd.DataFrame({'gndenergy':[np.power(np.abs(energy[1]-energy[0]),2)]})
filename = generate_file_name_hdf5(params)
## Complete path
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, gndstate_energy = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, gndenergy, filename1, key = key)

In [129]:
## Testing
gndenergy = load_matrix_from_hdf5(filename1, key = key)
print(gndenergy[0][0])

0.0022178302018838318


##### Entropy

In [130]:
entropy1 = driver.get_bipartite_entanglement(kets[0])

###### HDF5 Files

In [131]:
entropy = pd.DataFrame(entropy1)
filename = generate_file_name_hdf5(params, entropy = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, entropy = True)
append_data_to_hdf5(params, entropy, filename, key = 'entropy', entropy = True)

##### Correlation Matrix

In [132]:
hop_exp_val = np.zeros((params['L'], params['L']), dtype=complex)
for i in range(params['L']):
    for j in range(i, params['L']):
        b = driver.expr_builder()
        b.add_term("CD", np.array([i, j]), 1)  # Hopping operator between i and j            
        hop_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
        hop_exp_val[i, j] = driver.expectation(kets[0], hop_mpo, kets[0])
        hop_exp_val[j, i] = np.conjugate(hop_exp_val[i, j])

In [133]:
correl_matrix = pd.DataFrame(hop_exp_val)
filename = generate_file_name_hdf5(params, Correlation = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, Correlation = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

##### NumberOperator and Its Standard Deviation

In [134]:
std_val, std_nOp = 0, 0
_exp_nop = []
for i in range(params['L']):
    b = driver.expr_builder()
    b.add_term("N", np.array([i]), 1)
## Calculate <N>   
    nOp_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    impo = driver.get_identity_mpo()

    nOp_exp = driver.expectation(kets[0], nOp_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0])
    _exp_nop.append(nOp_exp)
## Calculate <N^2>
    b = driver.expr_builder()
    b.add_term("NN", np.repeat([i], 2), 1)
    
    NN_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    NN_exp = driver.expectation(kets[0], NN_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0]) 
## Calculate Standard Deviation
    std_val+=np.sqrt(NN_exp - (nOp_exp*nOp_exp))        
std_nOp = (std_val/params['L'])

###### Expectation value of Number Operator on each site

In [135]:
density_nop = pd.DataFrame(_exp_nop)
filename = generate_file_name_hdf5(params, expnop = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, expnop = True)
key = generate_unique_key(params, exp_nOp = True)
append_data_to_hdf5(params, density_nop, filename, key = key, expnop = True)

In [136]:
# key = generate_unique_key(params, exp_nOp = True)
# density_nop = load_matrix_from_hdf5(filename1, key = key)
# print(density_nop)

###### Standard Deviation

In [137]:
std_deviation_numberOp = pd.DataFrame({'std_nOp':[std_nOp]})
filename = generate_file_name_hdf5(params)
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, std_deviation_numberOp = True)

## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, std_deviation_numberOp, filename1, key = key)

####### Use this only for the first time when taking data for a theta #######
## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, std_deviation_numberOp, filename, key = key)

In [138]:
key = generate_unique_key(params, std_deviation_numberOp = True)
std_nop = load_matrix_from_hdf5(filename1, key = key)
print(std_nop[0][0])

(0.6429408817001884+1.9747708314194016e-19j)


In [139]:
with pd.HDFStore(filename1, 'r') as hdf:
    # List all keys in the file
    keys = hdf.keys()
    print("Keys in HDF5 file:")
    for key in keys:
        print(key)

Keys in HDF5 file:
/gndenergy_U7_0e01_t1_0e0
/std_deviation_numberOp_U7_0e01_t1_0e0


#### $U = 0.8$

In [140]:
start_time = time.time()
#########################################

params['u'] = 0.8
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4  + [200] * 4 + [250] * 4 + [300] * 4 + [350] * 4 + [400] * 4 + [450] * 4 + [500] * 4 + [550] * 4 + [600] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)

#########################################
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

Execution time: 158.31 seconds


In [141]:
kets = [driver.split_mps(mps, ir, tag="KET-%d" % ir) for ir in range(mps.nroots)]

##### First Excitation Gap

In [142]:
gndenergy = pd.DataFrame({'gndenergy':[np.power(np.abs(energy[1]-energy[0]),2)]})
filename = generate_file_name_hdf5(params)
## Complete path
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, gndstate_energy = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, gndenergy, filename1, key = key)

In [143]:
## Testing
gndenergy = load_matrix_from_hdf5(filename1, key = key)
print(gndenergy[0][0])

0.0006874561767174338


##### Entropy

In [144]:
entropy1 = driver.get_bipartite_entanglement(kets[0])

###### HDF5 Files

In [145]:
entropy = pd.DataFrame(entropy1)
filename = generate_file_name_hdf5(params, entropy = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, entropy = True)
append_data_to_hdf5(params, entropy, filename, key = 'entropy', entropy = True)

##### Correlation Matrix

In [146]:
hop_exp_val = np.zeros((params['L'], params['L']), dtype=complex)
for i in range(params['L']):
    for j in range(i, params['L']):
        b = driver.expr_builder()
        b.add_term("CD", np.array([i, j]), 1)  # Hopping operator between i and j            
        hop_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
        hop_exp_val[i, j] = driver.expectation(kets[0], hop_mpo, kets[0])
        hop_exp_val[j, i] = np.conjugate(hop_exp_val[i, j])

In [147]:
correl_matrix = pd.DataFrame(hop_exp_val)
filename = generate_file_name_hdf5(params, Correlation = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, Correlation = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

##### NumberOperator and Its Standard Deviation

In [148]:
std_val, std_nOp = 0, 0
_exp_nop = []
for i in range(params['L']):
    b = driver.expr_builder()
    b.add_term("N", np.array([i]), 1)
## Calculate <N>   
    nOp_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    impo = driver.get_identity_mpo()

    nOp_exp = driver.expectation(kets[0], nOp_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0])
    _exp_nop.append(nOp_exp)
## Calculate <N^2>
    b = driver.expr_builder()
    b.add_term("NN", np.repeat([i], 2), 1)
    
    NN_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    NN_exp = driver.expectation(kets[0], NN_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0]) 
## Calculate Standard Deviation
    std_val+=np.sqrt(NN_exp - (nOp_exp*nOp_exp))        
std_nOp = (std_val/params['L'])

###### Expectation value of Number Operator on each site

In [149]:
density_nop = pd.DataFrame(_exp_nop)
filename = generate_file_name_hdf5(params, expnop = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, expnop = True)
key = generate_unique_key(params, exp_nOp = True)
append_data_to_hdf5(params, density_nop, filename, key = key, expnop = True)

In [150]:
# key = generate_unique_key(params, exp_nOp = True)
# density_nop = load_matrix_from_hdf5(filename1, key = key)
# print(density_nop)

###### Standard Deviation

In [151]:
std_deviation_numberOp = pd.DataFrame({'std_nOp':[std_nOp]})
filename = generate_file_name_hdf5(params)
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, std_deviation_numberOp = True)

## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, std_deviation_numberOp, filename1, key = key)

####### Use this only for the first time when taking data for a theta #######
## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, std_deviation_numberOp, filename, key = key)

In [152]:
key = generate_unique_key(params, std_deviation_numberOp = True)
std_nop = load_matrix_from_hdf5(filename1, key = key)
print(std_nop[0][0])

(0.6384506898164473-8.47959028481768e-18j)


In [153]:
with pd.HDFStore(filename1, 'r') as hdf:
    # List all keys in the file
    keys = hdf.keys()
    print("Keys in HDF5 file:")
    for key in keys:
        print(key)

Keys in HDF5 file:
/gndenergy_U8_0e01_t1_0e0
/std_deviation_numberOp_U8_0e01_t1_0e0


#### $U = 0.9$

In [154]:
start_time = time.time()
#########################################

params['u'] = 0.9
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4 + [200] * 4 + [250] * 4 + [300] * 4 + [350] * 4 + [400] * 4 + [450] * 4 + [500] * 4 + [550] * 4 + [600] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)

#########################################
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

Execution time: 149.28 seconds


In [155]:
kets = [driver.split_mps(mps, ir, tag="KET-%d" % ir) for ir in range(mps.nroots)]

##### First Excitation Gap

In [156]:
gndenergy = pd.DataFrame({'gndenergy':[np.power(np.abs(energy[1]-energy[0]),2)]})
filename = generate_file_name_hdf5(params)
## Complete path
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, gndstate_energy = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, gndenergy, filename1, key = key)

In [157]:
## Testing
gndenergy = load_matrix_from_hdf5(filename1, key = key)
print(gndenergy[0][0])

0.001181550171635085


##### Entropy

In [158]:
entropy1 = driver.get_bipartite_entanglement(kets[0])

###### HDF5 Files

In [159]:
entropy = pd.DataFrame(entropy1)
filename = generate_file_name_hdf5(params, entropy = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, entropy = True)
append_data_to_hdf5(params, entropy, filename, key = 'entropy', entropy = True)

##### Correlation Matrix

In [160]:
hop_exp_val = np.zeros((params['L'], params['L']), dtype=complex)
for i in range(params['L']):
    for j in range(i, params['L']):
        b = driver.expr_builder()
        b.add_term("CD", np.array([i, j]), 1)  # Hopping operator between i and j            
        hop_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
        hop_exp_val[i, j] = driver.expectation(kets[0], hop_mpo, kets[0])
        hop_exp_val[j, i] = np.conjugate(hop_exp_val[i, j])

In [161]:
correl_matrix = pd.DataFrame(hop_exp_val)
filename = generate_file_name_hdf5(params, Correlation = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, Correlation = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

##### NumberOperator and Its Standard Deviation

In [162]:
std_val, std_nOp = 0, 0
_exp_nop = []
for i in range(params['L']):
    b = driver.expr_builder()
    b.add_term("N", np.array([i]), 1)
## Calculate <N>   
    nOp_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    impo = driver.get_identity_mpo()

    nOp_exp = driver.expectation(kets[0], nOp_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0])
    _exp_nop.append(nOp_exp)
## Calculate <N^2>
    b = driver.expr_builder()
    b.add_term("NN", np.repeat([i], 2), 1)
    
    NN_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    NN_exp = driver.expectation(kets[0], NN_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0]) 
## Calculate Standard Deviation
    std_val+=np.sqrt(NN_exp - (nOp_exp*nOp_exp))        
std_nOp = (std_val/params['L'])

###### Expectation value of Number Operator on each site

In [163]:
density_nop = pd.DataFrame(_exp_nop)
filename = generate_file_name_hdf5(params, expnop = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, expnop = True)
key = generate_unique_key(params, exp_nOp = True)
append_data_to_hdf5(params, density_nop, filename, key = key, expnop = True)

In [164]:
# key = generate_unique_key(params, exp_nOp = True)
# density_nop = load_matrix_from_hdf5(filename1, key = key)
# print(density_nop)

###### Standard Deviation

In [165]:
std_deviation_numberOp = pd.DataFrame({'std_nOp':[std_nOp]})
filename = generate_file_name_hdf5(params)
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, std_deviation_numberOp = True)

## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, std_deviation_numberOp, filename1, key = key)

####### Use this only for the first time when taking data for a theta #######
## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, std_deviation_numberOp, filename, key = key)

In [166]:
key = generate_unique_key(params, std_deviation_numberOp = True)
std_nop = load_matrix_from_hdf5(filename1, key = key)
print(std_nop[0][0])

(0.6304910276293414+1.7628975048141727e-17j)


In [167]:
with pd.HDFStore(filename1, 'r') as hdf:
    # List all keys in the file
    keys = hdf.keys()
    print("Keys in HDF5 file:")
    for key in keys:
        print(key)

Keys in HDF5 file:
/gndenergy_U9_0e01_t1_0e0
/std_deviation_numberOp_U9_0e01_t1_0e0


#### $U = 1.0$

In [168]:
start_time = time.time()
#########################################

params['u'] = 1
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4 + [200] * 4 + [250] * 4 + [300] * 4 + [350] * 4 + [400] * 4 + [450] * 4 + [500] * 4 + [550] * 4 + [600] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)

#########################################
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

Execution time: 148.51 seconds


In [169]:
kets = [driver.split_mps(mps, ir, tag="KET-%d" % ir) for ir in range(mps.nroots)]

##### First Excitation Gap

In [170]:
gndenergy = pd.DataFrame({'gndenergy':[np.power(np.abs(energy[1]-energy[0]),2)]})
filename = generate_file_name_hdf5(params)
## Complete path
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, gndstate_energy = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, gndenergy, filename1, key = key)

In [171]:
## Testing
gndenergy = load_matrix_from_hdf5(filename1, key = key)
print(gndenergy[0][0])

0.0018682493306533265


##### Entropy

In [172]:
entropy1 = driver.get_bipartite_entanglement(kets[0])

###### HDF5 Files

In [173]:
entropy = pd.DataFrame(entropy1)
filename = generate_file_name_hdf5(params, entropy = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, entropy = True)
append_data_to_hdf5(params, entropy, filename, key = 'entropy', entropy = True)

##### Correlation Matrix

In [174]:
hop_exp_val = np.zeros((params['L'], params['L']), dtype=complex)
for i in range(params['L']):
    for j in range(i, params['L']):
        b = driver.expr_builder()
        b.add_term("CD", np.array([i, j]), 1)  # Hopping operator between i and j            
        hop_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
        hop_exp_val[i, j] = driver.expectation(kets[0], hop_mpo, kets[0])
        hop_exp_val[j, i] = np.conjugate(hop_exp_val[i, j])

In [175]:
correl_matrix = pd.DataFrame(hop_exp_val)
filename = generate_file_name_hdf5(params, Correlation = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, Correlation = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

##### NumberOperator and Its Standard Deviation

In [176]:
std_val, std_nOp = 0, 0
_exp_nop = []
for i in range(params['L']):
    b = driver.expr_builder()
    b.add_term("N", np.array([i]), 1)
## Calculate <N>   
    nOp_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    impo = driver.get_identity_mpo()

    nOp_exp = driver.expectation(kets[0], nOp_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0])
    _exp_nop.append(nOp_exp)
## Calculate <N^2>
    b = driver.expr_builder()
    b.add_term("NN", np.repeat([i], 2), 1)
    
    NN_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    NN_exp = driver.expectation(kets[0], NN_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0]) 
## Calculate Standard Deviation
    std_val+=np.sqrt(NN_exp - (nOp_exp*nOp_exp))        
std_nOp = (std_val/params['L'])

###### Expectation value of Number Operator on each site

In [177]:
density_nop = pd.DataFrame(_exp_nop)
filename = generate_file_name_hdf5(params, expnop = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, expnop = True)
key = generate_unique_key(params, exp_nOp = True)
append_data_to_hdf5(params, density_nop, filename, key = key, expnop = True)

In [178]:
# key = generate_unique_key(params, exp_nOp = True)
# density_nop = load_matrix_from_hdf5(filename1, key = key)
# print(density_nop)

###### Standard Deviation

In [179]:
std_deviation_numberOp = pd.DataFrame({'std_nOp':[std_nOp]})
filename = generate_file_name_hdf5(params)
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, std_deviation_numberOp = True)

## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, std_deviation_numberOp, filename1, key = key)

####### Use this only for the first time when taking data for a theta #######
## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, std_deviation_numberOp, filename, key = key)

In [180]:
key = generate_unique_key(params, std_deviation_numberOp = True)
std_nop = load_matrix_from_hdf5(filename1, key = key)
print(std_nop[0][0])

(0.6173579594599204-2.3371514696518006e-19j)


In [181]:
with pd.HDFStore(filename1, 'r') as hdf:
    # List all keys in the file
    keys = hdf.keys()
    print("Keys in HDF5 file:")
    for key in keys:
        print(key)

Keys in HDF5 file:
/gndenergy_U1_0e0_t1_0e0
/std_deviation_numberOp_U1_0e0_t1_0e0


#### $U = 1.1$

In [182]:
start_time = time.time()
#########################################

params['u'] = 1.1
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4 + [200] * 4 + [250] * 4 + [300] * 4 + [350] * 4 + [400] * 4 + [450] * 4 + [500] * 4 + [550] * 4 + [600] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)

#########################################
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

Execution time: 150.56 seconds


In [183]:
kets = [driver.split_mps(mps, ir, tag="KET-%d" % ir) for ir in range(mps.nroots)]

##### First Excitation Gap

In [184]:
gndenergy = pd.DataFrame({'gndenergy':[np.power(np.abs(energy[1]-energy[0]),2)]})
filename = generate_file_name_hdf5(params)
## Complete path
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, gndstate_energy = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, gndenergy, filename1, key = key)

In [185]:
## Testing
gndenergy = load_matrix_from_hdf5(filename1, key = key)
print(gndenergy[0][0])

0.0009609495409292019


##### Entropy

In [186]:
entropy1 = driver.get_bipartite_entanglement(kets[0])

###### HDF5 Files

In [187]:
entropy = pd.DataFrame(entropy1)
filename = generate_file_name_hdf5(params, entropy = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, entropy = True)
create_folders_and_store_data(params, entropy, filename1, key = 'entropy', entropy = True)

##### Correlation Matrix

In [188]:
hop_exp_val = np.zeros((params['L'], params['L']), dtype=complex)
for i in range(params['L']):
    for j in range(i, params['L']):
        b = driver.expr_builder()
        b.add_term("CD", np.array([i, j]), 1)  # Hopping operator between i and j            
        hop_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
        hop_exp_val[i, j] = driver.expectation(kets[0], hop_mpo, kets[0])
        hop_exp_val[j, i] = np.conjugate(hop_exp_val[i, j])

In [189]:
correl_matrix = pd.DataFrame(hop_exp_val)
filename = generate_file_name_hdf5(params, Correlation = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, Correlation = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

##### NumberOperator and Its Standard Deviation

In [190]:
std_val, std_nOp = 0, 0
_exp_nop = []
for i in range(params['L']):
    b = driver.expr_builder()
    b.add_term("N", np.array([i]), 1)
## Calculate <N>   
    nOp_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    impo = driver.get_identity_mpo()

    nOp_exp = driver.expectation(kets[0], nOp_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0])
    _exp_nop.append(nOp_exp)
## Calculate <N^2>
    b = driver.expr_builder()
    b.add_term("NN", np.repeat([i], 2), 1)
    
    NN_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    NN_exp = driver.expectation(kets[0], NN_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0]) 
## Calculate Standard Deviation
    std_val+=np.sqrt(NN_exp - (nOp_exp*nOp_exp))        
std_nOp = (std_val/params['L'])

###### Expectation value of Number Operator on each site

In [191]:
density_nop = pd.DataFrame(_exp_nop)
filename = generate_file_name_hdf5(params, expnop = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, expnop = True)
key = generate_unique_key(params, exp_nOp = True)
append_data_to_hdf5(params, density_nop, filename, key = key, expnop = True)

In [192]:
# key = generate_unique_key(params, exp_nOp = True)
# density_nop = load_matrix_from_hdf5(filename1, key = key)
# print(density_nop)

###### Standard Deviation

In [193]:
std_deviation_numberOp = pd.DataFrame({'std_nOp':[std_nOp]})
filename = generate_file_name_hdf5(params)
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, std_deviation_numberOp = True)

## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, std_deviation_numberOp, filename1, key = key)

####### Use this only for the first time when taking data for a theta #######
## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, std_deviation_numberOp, filename, key = key)

In [194]:
key = generate_unique_key(params, std_deviation_numberOp = True)
std_nop = load_matrix_from_hdf5(filename1, key = key)
print(std_nop[0][0])

(0.611390057297839-3.111390381940938e-18j)


In [195]:
with pd.HDFStore(filename1, 'r') as hdf:
    # List all keys in the file
    keys = hdf.keys()
    print("Keys in HDF5 file:")
    for key in keys:
        print(key)

Keys in HDF5 file:
/gndenergy_U1_1e0_t1_0e0
/std_deviation_numberOp_U1_1e0_t1_0e0


#### $U = 1.2$

In [196]:
start_time = time.time()
#########################################

params['u'] = 1.2
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4 + [200] * 4 + [250] * 4 + [300] * 4 + [350] * 4 + [400] * 4 + [450] * 4 + [500] * 4 + [550] * 4 + [600] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)

#########################################
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

Execution time: 154.21 seconds


In [197]:
kets = [driver.split_mps(mps, ir, tag="KET-%d" % ir) for ir in range(mps.nroots)]

##### First Excitation Gap

In [198]:
gndenergy = pd.DataFrame({'gndenergy':[np.power(np.abs(energy[1]-energy[0]),2)]})
filename = generate_file_name_hdf5(params)
## Complete path
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, gndstate_energy = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, gndenergy, filename1, key = key)

In [199]:
## Testing
gndenergy = load_matrix_from_hdf5(filename1, key = key)
print(gndenergy[0][0])

0.000871812134525353


##### Entropy

In [200]:
entropy1 = driver.get_bipartite_entanglement(kets[0])

###### HDF5 Files

In [201]:
entropy = pd.DataFrame(entropy1)
filename = generate_file_name_hdf5(params, entropy = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, entropy = True)
append_data_to_hdf5(params, entropy, filename, key = 'entropy', entropy = True)

##### Correlation Matrix

In [202]:
hop_exp_val = np.zeros((params['L'], params['L']), dtype=complex)
for i in range(params['L']):
    for j in range(i, params['L']):
        b = driver.expr_builder()
        b.add_term("CD", np.array([i, j]), 1)  # Hopping operator between i and j            
        hop_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
        hop_exp_val[i, j] = driver.expectation(kets[0], hop_mpo, kets[0])
        hop_exp_val[j, i] = np.conjugate(hop_exp_val[i, j])

In [203]:
correl_matrix = pd.DataFrame(hop_exp_val)
filename = generate_file_name_hdf5(params, Correlation = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, Correlation = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

##### NumberOperator and Its Standard Deviation

In [204]:
std_val, std_nOp = 0, 0
_exp_nop = []
for i in range(params['L']):
    b = driver.expr_builder()
    b.add_term("N", np.array([i]), 1)
## Calculate <N>   
    nOp_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    impo = driver.get_identity_mpo()

    nOp_exp = driver.expectation(kets[0], nOp_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0])
    _exp_nop.append(nOp_exp)
## Calculate <N^2>
    b = driver.expr_builder()
    b.add_term("NN", np.repeat([i], 2), 1)
    
    NN_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    NN_exp = driver.expectation(kets[0], NN_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0]) 
## Calculate Standard Deviation
    std_val+=np.sqrt(NN_exp - (nOp_exp*nOp_exp))        
std_nOp = (std_val/params['L'])

###### Expectation value of Number Operator on each site

In [205]:
density_nop = pd.DataFrame(_exp_nop)
filename = generate_file_name_hdf5(params, expnop = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, expnop = True)
key = generate_unique_key(params, exp_nOp = True)
append_data_to_hdf5(params, density_nop, filename, key = key, expnop = True)

In [206]:
# key = generate_unique_key(params, exp_nOp = True)
# density_nop = load_matrix_from_hdf5(filename1, key = key)
# print(density_nop)

###### Standard Deviation

In [207]:
std_deviation_numberOp = pd.DataFrame({'std_nOp':[std_nOp]})
filename = generate_file_name_hdf5(params)
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, std_deviation_numberOp = True)

## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, std_deviation_numberOp, filename1, key = key)

####### Use this only for the first time when taking data for a theta #######
## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, std_deviation_numberOp, filename, key = key)

In [208]:
key = generate_unique_key(params, std_deviation_numberOp = True)
std_nop = load_matrix_from_hdf5(filename1, key = key)
print(std_nop[0][0])

(0.6094379917308969-2.6704972409568944e-19j)


In [209]:
with pd.HDFStore(filename1, 'r') as hdf:
    # List all keys in the file
    keys = hdf.keys()
    print("Keys in HDF5 file:")
    for key in keys:
        print(key)

Keys in HDF5 file:
/gndenergy_U1_2e0_t1_0e0
/std_deviation_numberOp_U1_2e0_t1_0e0


#### $U = 1.3$

In [210]:
start_time = time.time()
#########################################


params['u'] = 1.3
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4 + [200] * 4 + [250] * 4 + [300] * 4 + [350] * 4 + [400] * 4 + [450] * 4 + [500] * 4 + [550] * 4 + [600] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)

#########################################
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

Execution time: 149.02 seconds


In [211]:
kets = [driver.split_mps(mps, ir, tag="KET-%d" % ir) for ir in range(mps.nroots)]

##### First Excitation Gap

In [212]:
gndenergy = pd.DataFrame({'gndenergy':[np.power(np.abs(energy[1]-energy[0]),2)]})
filename = generate_file_name_hdf5(params)
## Complete path
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, gndstate_energy = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, gndenergy, filename1, key = key)

In [213]:
## Testing
gndenergy = load_matrix_from_hdf5(filename1, key = key)
print(gndenergy[0][0])

0.0004280993120023111


##### Entropy

In [214]:
entropy1 = driver.get_bipartite_entanglement(kets[0])

###### HDF5 Files

In [215]:
entropy = pd.DataFrame(entropy1)
filename = generate_file_name_hdf5(params, entropy = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, entropy = True)
append_data_to_hdf5(params, entropy, filename, key = 'entropy', entropy = True)

##### Correlation Matrix

In [216]:
hop_exp_val = np.zeros((params['L'], params['L']), dtype=complex)
for i in range(params['L']):
    for j in range(i, params['L']):
        b = driver.expr_builder()
        b.add_term("CD", np.array([i, j]), 1)  # Hopping operator between i and j            
        hop_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
        hop_exp_val[i, j] = driver.expectation(kets[0], hop_mpo, kets[0])
        hop_exp_val[j, i] = np.conjugate(hop_exp_val[i, j])

In [217]:
correl_matrix = pd.DataFrame(hop_exp_val)
filename = generate_file_name_hdf5(params, Correlation = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, Correlation = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

##### NumberOperator and Its Standard Deviation

In [218]:
std_val, std_nOp = 0, 0
_exp_nop = []
for i in range(params['L']):
    b = driver.expr_builder()
    b.add_term("N", np.array([i]), 1)
## Calculate <N>   
    nOp_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    impo = driver.get_identity_mpo()

    nOp_exp = driver.expectation(kets[0], nOp_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0])
    _exp_nop.append(nOp_exp)
## Calculate <N^2>
    b = driver.expr_builder()
    b.add_term("NN", np.repeat([i], 2), 1)
    
    NN_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    NN_exp = driver.expectation(kets[0], NN_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0]) 
## Calculate Standard Deviation
    std_val+=np.sqrt(NN_exp - (nOp_exp*nOp_exp))        
std_nOp = (std_val/params['L'])

###### Expectation value of Number Operator on each site

In [219]:
density_nop = pd.DataFrame(_exp_nop)
filename = generate_file_name_hdf5(params, expnop = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, expnop = True)
key = generate_unique_key(params, exp_nOp = True)
append_data_to_hdf5(params, density_nop, filename, key = key, expnop = True)

In [220]:
# key = generate_unique_key(params, exp_nOp = True)
# density_nop = load_matrix_from_hdf5(filename1, key = key)
# print(density_nop)

###### Standard Deviation

In [221]:
std_deviation_numberOp = pd.DataFrame({'std_nOp':[std_nOp]})
filename = generate_file_name_hdf5(params)
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, std_deviation_numberOp = True)

## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, std_deviation_numberOp, filename1, key = key)

####### Use this only for the first time when taking data for a theta #######
## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, std_deviation_numberOp, filename, key = key)

In [222]:
key = generate_unique_key(params, std_deviation_numberOp = True)
std_nop = load_matrix_from_hdf5(filename1, key = key)
print(std_nop[0][0])

(0.5938832117125649-5.1387681117518025e-18j)


In [223]:
with pd.HDFStore(filename1, 'r') as hdf:
    # List all keys in the file
    keys = hdf.keys()
    print("Keys in HDF5 file:")
    for key in keys:
        print(key)

Keys in HDF5 file:
/gndenergy_U1_3e0_t1_0e0
/std_deviation_numberOp_U1_3e0_t1_0e0


#### $U = 1.4$

In [224]:
start_time = time.time()
#########################################

params['u'] = 1.4
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4 + [200] * 4 + [250] * 4 + [300] * 4 + [350] * 4 + [400] * 4 + [450] * 4 + [500] * 4 + [550] * 4 + [600] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)

#########################################
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

Execution time: 160.41 seconds


In [225]:
kets = [driver.split_mps(mps, ir, tag="KET-%d" % ir) for ir in range(mps.nroots)]

##### First Excitation Gap

In [226]:
gndenergy = pd.DataFrame({'gndenergy':[np.power(np.abs(energy[1]-energy[0]),2)]})
filename = generate_file_name_hdf5(params)
## Complete path
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, gndstate_energy = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, gndenergy, filename1, key = key)

In [227]:
## Testing
gndenergy = load_matrix_from_hdf5(filename1, key = key)
print(gndenergy[0][0])

0.0004684071129829936


##### Entropy

In [228]:
entropy1 = driver.get_bipartite_entanglement(kets[0])

###### HDF5 Files

In [229]:
entropy = pd.DataFrame(entropy1)
filename = generate_file_name_hdf5(params, entropy = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, entropy = True)
append_data_to_hdf5(params, entropy, filename, key = 'entropy', entropy = True)

##### Correlation Matrix

In [230]:
hop_exp_val = np.zeros((params['L'], params['L']), dtype=complex)
for i in range(params['L']):
    for j in range(i, params['L']):
        b = driver.expr_builder()
        b.add_term("CD", np.array([i, j]), 1)  # Hopping operator between i and j            
        hop_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
        hop_exp_val[i, j] = driver.expectation(kets[0], hop_mpo, kets[0])
        hop_exp_val[j, i] = np.conjugate(hop_exp_val[i, j])

In [231]:
correl_matrix = pd.DataFrame(hop_exp_val)
filename = generate_file_name_hdf5(params, Correlation = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, Correlation = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

##### NumberOperator and Its Standard Deviation

In [232]:
std_val, std_nOp = 0, 0
_exp_nop = []
for i in range(params['L']):
    b = driver.expr_builder()
    b.add_term("N", np.array([i]), 1)
## Calculate <N>   
    nOp_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    impo = driver.get_identity_mpo()

    nOp_exp = driver.expectation(kets[0], nOp_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0])
    _exp_nop.append(nOp_exp)
## Calculate <N^2>
    b = driver.expr_builder()
    b.add_term("NN", np.repeat([i], 2), 1)
    
    NN_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    NN_exp = driver.expectation(kets[0], NN_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0]) 
## Calculate Standard Deviation
    std_val+=np.sqrt(NN_exp - (nOp_exp*nOp_exp))        
std_nOp = (std_val/params['L'])

###### Expectation value of Number Operator on each site

In [233]:
density_nop = pd.DataFrame(_exp_nop)
filename = generate_file_name_hdf5(params, expnop = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, expnop = True)
key = generate_unique_key(params, exp_nOp = True)
append_data_to_hdf5(params, density_nop, filename, key = key, expnop = True)

In [234]:
# key = generate_unique_key(params, exp_nOp = True)
# density_nop = load_matrix_from_hdf5(filename1, key = key)
# print(density_nop)

###### Standard Deviation

In [235]:
std_deviation_numberOp = pd.DataFrame({'std_nOp':[std_nOp]})
filename = generate_file_name_hdf5(params)
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, std_deviation_numberOp = True)

## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, std_deviation_numberOp, filename1, key = key)

####### Use this only for the first time when taking data for a theta #######
## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, std_deviation_numberOp, filename, key = key)

In [236]:
key = generate_unique_key(params, std_deviation_numberOp = True)
std_nop = load_matrix_from_hdf5(filename1, key = key)
print(std_nop[0][0])

(0.5935187118152022+4.027739447190763e-20j)


In [237]:
with pd.HDFStore(filename1, 'r') as hdf:
    # List all keys in the file
    keys = hdf.keys()
    print("Keys in HDF5 file:")
    for key in keys:
        print(key)

Keys in HDF5 file:
/gndenergy_U1_4e0_t1_0e0
/std_deviation_numberOp_U1_4e0_t1_0e0


#### $U = 1.5$

In [238]:
start_time = time.time()
#########################################

params['u'] = 1.5
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4 + [200] * 4 + [250] * 4 + [300] * 4 + [350] * 4 + [400] * 4 + [450] * 4 + [500] * 4 + [550] * 4 + [600] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)

#########################################
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

Execution time: 158.66 seconds


In [239]:
kets = [driver.split_mps(mps, ir, tag="KET-%d" % ir) for ir in range(mps.nroots)]

##### First Excitation Gap

In [240]:
gndenergy = pd.DataFrame({'gndenergy':[np.power(np.abs(energy[1]-energy[0]),2)]})
filename = generate_file_name_hdf5(params)
## Complete path
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, gndstate_energy = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, gndenergy, filename1, key = key)

In [241]:
## Testing
gndenergy = load_matrix_from_hdf5(filename1, key = key)
print(gndenergy[0][0])

0.00033894732325986574


##### Entropy

In [242]:
entropy1 = driver.get_bipartite_entanglement(kets[0])

###### HDF5 Files

In [243]:
entropy = pd.DataFrame(entropy1)
filename = generate_file_name_hdf5(params, entropy = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, entropy = True)
append_data_to_hdf5(params, entropy, filename, key = 'entropy', entropy = True)

##### Correlation Matrix

In [244]:
hop_exp_val = np.zeros((params['L'], params['L']), dtype=complex)
for i in range(params['L']):
    for j in range(i, params['L']):
        b = driver.expr_builder()
        b.add_term("CD", np.array([i, j]), 1)  # Hopping operator between i and j            
        hop_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
        hop_exp_val[i, j] = driver.expectation(kets[0], hop_mpo, kets[0])
        hop_exp_val[j, i] = np.conjugate(hop_exp_val[i, j])

In [245]:
correl_matrix = pd.DataFrame(hop_exp_val)
filename = generate_file_name_hdf5(params, Correlation = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, Correlation = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

##### NumberOperator and Its Standard Deviation

In [246]:
std_val, std_nOp = 0, 0
_exp_nop = []
for i in range(params['L']):
    b = driver.expr_builder()
    b.add_term("N", np.array([i]), 1)
## Calculate <N>   
    nOp_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    impo = driver.get_identity_mpo()

    nOp_exp = driver.expectation(kets[0], nOp_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0])
    _exp_nop.append(nOp_exp)
## Calculate <N^2>
    b = driver.expr_builder()
    b.add_term("NN", np.repeat([i], 2), 1)
    
    NN_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    NN_exp = driver.expectation(kets[0], NN_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0]) 
## Calculate Standard Deviation
    std_val+=np.sqrt(NN_exp - (nOp_exp*nOp_exp))        
std_nOp = (std_val/params['L'])

###### Expectation value of Number Operator on each site

In [247]:
density_nop = pd.DataFrame(_exp_nop)
filename = generate_file_name_hdf5(params, expnop = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, expnop = True)
key = generate_unique_key(params, exp_nOp = True)
append_data_to_hdf5(params, density_nop, filename, key = key, expnop = True)

In [248]:
# key = generate_unique_key(params, exp_nOp = True)
# density_nop = load_matrix_from_hdf5(filename1, key = key)
# print(density_nop)

###### Standard Deviation

In [249]:
std_deviation_numberOp = pd.DataFrame({'std_nOp':[std_nOp]})
filename = generate_file_name_hdf5(params)
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, std_deviation_numberOp = True)

## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, std_deviation_numberOp, filename1, key = key)

####### Use this only for the first time when taking data for a theta #######
## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, std_deviation_numberOp, filename, key = key)

In [250]:
key = generate_unique_key(params, std_deviation_numberOp = True)
std_nop = load_matrix_from_hdf5(filename1, key = key)
print(std_nop[0][0])

(0.5836202271963725+7.941900741090982e-19j)


In [251]:
with pd.HDFStore(filename1, 'r') as hdf:
    # List all keys in the file
    keys = hdf.keys()
    print("Keys in HDF5 file:")
    for key in keys:
        print(key)

Keys in HDF5 file:
/gndenergy_U1_5e0_t1_0e0
/std_deviation_numberOp_U1_5e0_t1_0e0


#### $U = 1.6$

In [252]:
start_time = time.time()
#########################################


params['u'] = 1.6
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4 + [200] * 4 + [250] * 4 + [300] * 4 + [350] * 4 + [400] * 4 + [450] * 4 + [500] * 4 + [550] * 4 + [600] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)

#########################################
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

Execution time: 161.01 seconds


In [253]:
kets = [driver.split_mps(mps, ir, tag="KET-%d" % ir) for ir in range(mps.nroots)]

##### First Excitation Gap

In [254]:
gndenergy = pd.DataFrame({'gndenergy':[np.power(np.abs(energy[1]-energy[0]),2)]})
filename = generate_file_name_hdf5(params)
## Complete path
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, gndstate_energy = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, gndenergy, filename1, key = key)

In [255]:
## Testing
gndenergy = load_matrix_from_hdf5(filename1, key = key)
print(gndenergy[0][0])

0.0002026018803580361


##### Entropy

In [256]:
entropy1 = driver.get_bipartite_entanglement(kets[0])

###### HDF5 Files

In [257]:
entropy = pd.DataFrame(entropy1)
filename = generate_file_name_hdf5(params, entropy = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, entropy = True)
append_data_to_hdf5(params, entropy, filename, key = 'entropy', entropy = True)

##### Correlation Matrix

In [258]:
hop_exp_val = np.zeros((params['L'], params['L']), dtype=complex)
for i in range(params['L']):
    for j in range(i, params['L']):
        b = driver.expr_builder()
        b.add_term("CD", np.array([i, j]), 1)  # Hopping operator between i and j            
        hop_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
        hop_exp_val[i, j] = driver.expectation(kets[0], hop_mpo, kets[0])
        hop_exp_val[j, i] = np.conjugate(hop_exp_val[i, j])

In [259]:
correl_matrix = pd.DataFrame(hop_exp_val)
filename = generate_file_name_hdf5(params, Correlation = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, Correlation = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

##### NumberOperator and Its Standard Deviation

In [260]:
std_val, std_nOp = 0, 0
_exp_nop = []
for i in range(params['L']):
    b = driver.expr_builder()
    b.add_term("N", np.array([i]), 1)
## Calculate <N>   
    nOp_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    impo = driver.get_identity_mpo()

    nOp_exp = driver.expectation(kets[0], nOp_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0])
    _exp_nop.append(nOp_exp)
## Calculate <N^2>
    b = driver.expr_builder()
    b.add_term("NN", np.repeat([i], 2), 1)
    
    NN_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    NN_exp = driver.expectation(kets[0], NN_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0]) 
## Calculate Standard Deviation
    std_val+=np.sqrt(NN_exp - (nOp_exp*nOp_exp))        
std_nOp = (std_val/params['L'])

###### Expectation value of Number Operator on each site

In [261]:
density_nop = pd.DataFrame(_exp_nop)
filename = generate_file_name_hdf5(params, expnop = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, expnop = True)
key = generate_unique_key(params, exp_nOp = True)
append_data_to_hdf5(params, density_nop, filename, key = key, expnop = True)

In [262]:
# key = generate_unique_key(params, exp_nOp = True)
# density_nop = load_matrix_from_hdf5(filename1, key = key)
# print(density_nop)

###### Standard Deviation

In [263]:
std_deviation_numberOp = pd.DataFrame({'std_nOp':[std_nOp]})
filename = generate_file_name_hdf5(params)
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, std_deviation_numberOp = True)

## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, std_deviation_numberOp, filename1, key = key)

####### Use this only for the first time when taking data for a theta #######
## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, std_deviation_numberOp, filename, key = key)

In [264]:
key = generate_unique_key(params, std_deviation_numberOp = True)
std_nop = load_matrix_from_hdf5(filename1, key = key)
print(std_nop[0][0])

(0.5701680934242517+9.941028196491671e-18j)


In [265]:
with pd.HDFStore(filename1, 'r') as hdf:
    # List all keys in the file
    keys = hdf.keys()
    print("Keys in HDF5 file:")
    for key in keys:
        print(key)

Keys in HDF5 file:
/gndenergy_U1_6e0_t1_0e0
/std_deviation_numberOp_U1_6e0_t1_0e0


#### $U = 1.7$

In [266]:
start_time = time.time()
#########################################


params['u'] = 1.7
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4 + [200] * 4 + [250] * 4 + [300] * 4 + [350] * 4 + [400] * 4 + [450] * 4 + [500] * 4 + [550] * 4 + [600] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)

#########################################
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

Execution time: 158.24 seconds


In [267]:
kets = [driver.split_mps(mps, ir, tag="KET-%d" % ir) for ir in range(mps.nroots)]

##### First Excitation Gap

In [268]:
gndenergy = pd.DataFrame({'gndenergy':[np.power(np.abs(energy[1]-energy[0]),2)]})
filename = generate_file_name_hdf5(params)
## Complete path
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, gndstate_energy = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, gndenergy, filename1, key = key)

In [269]:
## Testing
gndenergy = load_matrix_from_hdf5(filename1, key = key)
print(gndenergy[0][0])

0.00023298896301817055


##### Entropy

In [270]:
entropy1 = driver.get_bipartite_entanglement(kets[0])

###### HDF5 Files

In [271]:
entropy = pd.DataFrame(entropy1)
filename = generate_file_name_hdf5(params, entropy = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, entropy = True)
append_data_to_hdf5(params, entropy, filename, key = 'entropy', entropy = True)

##### Correlation Matrix

In [272]:
hop_exp_val = np.zeros((params['L'], params['L']), dtype=complex)
for i in range(params['L']):
    for j in range(i, params['L']):
        b = driver.expr_builder()
        b.add_term("CD", np.array([i, j]), 1)  # Hopping operator between i and j            
        hop_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
        hop_exp_val[i, j] = driver.expectation(kets[0], hop_mpo, kets[0])
        hop_exp_val[j, i] = np.conjugate(hop_exp_val[i, j])

In [273]:
correl_matrix = pd.DataFrame(hop_exp_val)
filename = generate_file_name_hdf5(params, Correlation = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, Correlation = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

##### NumberOperator and Its Standard Deviation

In [274]:
std_val, std_nOp = 0, 0
_exp_nop = []
for i in range(params['L']):
    b = driver.expr_builder()
    b.add_term("N", np.array([i]), 1)
## Calculate <N>   
    nOp_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    impo = driver.get_identity_mpo()

    nOp_exp = driver.expectation(kets[0], nOp_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0])
    _exp_nop.append(nOp_exp)
## Calculate <N^2>
    b = driver.expr_builder()
    b.add_term("NN", np.repeat([i], 2), 1)
    
    NN_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    NN_exp = driver.expectation(kets[0], NN_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0]) 
## Calculate Standard Deviation
    std_val+=np.sqrt(NN_exp - (nOp_exp*nOp_exp))        
std_nOp = (std_val/params['L'])

###### Expectation value of Number Operator on each site

In [275]:
density_nop = pd.DataFrame(_exp_nop)
filename = generate_file_name_hdf5(params, expnop = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, expnop = True)
key = generate_unique_key(params, exp_nOp = True)
append_data_to_hdf5(params, density_nop, filename, key = key, expnop = True)

In [276]:
# key = generate_unique_key(params, exp_nOp = True)
# density_nop = load_matrix_from_hdf5(filename1, key = key)
# print(density_nop)

###### Standard Deviation

In [277]:
std_deviation_numberOp = pd.DataFrame({'std_nOp':[std_nOp]})
filename = generate_file_name_hdf5(params)
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, std_deviation_numberOp = True)

## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, std_deviation_numberOp, filename1, key = key)

####### Use this only for the first time when taking data for a theta #######
## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, std_deviation_numberOp, filename, key = key)

In [278]:
key = generate_unique_key(params, std_deviation_numberOp = True)
std_nop = load_matrix_from_hdf5(filename1, key = key)
print(std_nop[0][0])

(0.5641223016780018-1.1384149974646664e-18j)


In [279]:
with pd.HDFStore(filename1, 'r') as hdf:
    # List all keys in the file
    keys = hdf.keys()
    print("Keys in HDF5 file:")
    for key in keys:
        print(key)

Keys in HDF5 file:
/gndenergy_U1_7e0_t1_0e0
/std_deviation_numberOp_U1_7e0_t1_0e0


#### $U = 1.8$

In [280]:
start_time = time.time()
#########################################

params['u'] = 1.8
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4 + [200] * 4 + [250] * 4 + [300] * 4 + [350] * 4 + [400] * 4 + [450] * 4 + [500] * 4 + [550] * 4 + [600] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)

#########################################
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

Execution time: 154.31 seconds


In [281]:
kets = [driver.split_mps(mps, ir, tag="KET-%d" % ir) for ir in range(mps.nroots)]

##### First Excitation Gap

In [282]:
gndenergy = pd.DataFrame({'gndenergy':[np.power(np.abs(energy[1]-energy[0]),2)]})
filename = generate_file_name_hdf5(params)
## Complete path
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, gndstate_energy = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, gndenergy, filename1, key = key)

In [283]:
## Testing
gndenergy = load_matrix_from_hdf5(filename1, key = key)
print(gndenergy[0][0])

0.00024372116306419043


##### Entropy

In [284]:
entropy1 = driver.get_bipartite_entanglement(kets[0])

###### HDF5 Files

In [285]:
entropy = pd.DataFrame(entropy1)
filename = generate_file_name_hdf5(params, entropy = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, entropy = True)
append_data_to_hdf5(params, entropy, filename, key = 'entropy', entropy = True)

##### Correlation Matrix

In [286]:
hop_exp_val = np.zeros((params['L'], params['L']), dtype=complex)
for i in range(params['L']):
    for j in range(i, params['L']):
        b = driver.expr_builder()
        b.add_term("CD", np.array([i, j]), 1)  # Hopping operator between i and j            
        hop_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
        hop_exp_val[i, j] = driver.expectation(kets[0], hop_mpo, kets[0])
        hop_exp_val[j, i] = np.conjugate(hop_exp_val[i, j])

In [287]:
correl_matrix = pd.DataFrame(hop_exp_val)
filename = generate_file_name_hdf5(params, Correlation = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, Correlation = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

##### NumberOperator and Its Standard Deviation

In [288]:
std_val, std_nOp = 0, 0
_exp_nop = []
for i in range(params['L']):
    b = driver.expr_builder()
    b.add_term("N", np.array([i]), 1)
## Calculate <N>   
    nOp_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    impo = driver.get_identity_mpo()

    nOp_exp = driver.expectation(kets[0], nOp_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0])
    _exp_nop.append(nOp_exp)
## Calculate <N^2>
    b = driver.expr_builder()
    b.add_term("NN", np.repeat([i], 2), 1)
    
    NN_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    NN_exp = driver.expectation(kets[0], NN_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0]) 
## Calculate Standard Deviation
    std_val+=np.sqrt(NN_exp - (nOp_exp*nOp_exp))        
std_nOp = (std_val/params['L'])

###### Expectation value of Number Operator on each site

In [289]:
density_nop = pd.DataFrame(_exp_nop)
filename = generate_file_name_hdf5(params, expnop = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, expnop = True)
key = generate_unique_key(params, exp_nOp = True)
append_data_to_hdf5(params, density_nop, filename, key = key, expnop = True)

In [290]:
# key = generate_unique_key(params, exp_nOp = True)
# density_nop = load_matrix_from_hdf5(filename1, key = key)
# print(density_nop)

###### Standard Deviation

In [291]:
std_deviation_numberOp = pd.DataFrame({'std_nOp':[std_nOp]})
filename = generate_file_name_hdf5(params)
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, std_deviation_numberOp = True)

## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, std_deviation_numberOp, filename1, key = key)

####### Use this only for the first time when taking data for a theta #######
## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, std_deviation_numberOp, filename, key = key)

In [292]:
key = generate_unique_key(params, std_deviation_numberOp = True)
std_nop = load_matrix_from_hdf5(filename1, key = key)
print(std_nop[0][0])

(0.5583183556971124-1.6068710372597941e-18j)


In [293]:
with pd.HDFStore(filename1, 'r') as hdf:
    # List all keys in the file
    keys = hdf.keys()
    print("Keys in HDF5 file:")
    for key in keys:
        print(key)

Keys in HDF5 file:
/gndenergy_U1_8e0_t1_0e0
/std_deviation_numberOp_U1_8e0_t1_0e0


#### $U = 1.9$

In [294]:
start_time = time.time()
#########################################

params['u'] = 1.9
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4 + [200] * 4 + [250] * 4 + [300] * 4 + [350] * 4 + [400] * 4 + [450] * 4 + [500] * 4 + [550] * 4 + [600] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)

#########################################
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

Execution time: 142.54 seconds


In [295]:
kets = [driver.split_mps(mps, ir, tag="KET-%d" % ir) for ir in range(mps.nroots)]

##### First Excitation Gap

In [296]:
gndenergy = pd.DataFrame({'gndenergy':[np.power(np.abs(energy[1]-energy[0]),2)]})
filename = generate_file_name_hdf5(params)
## Complete path
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, gndstate_energy = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, gndenergy, filename1, key = key)

In [297]:
## Testing
gndenergy = load_matrix_from_hdf5(filename1, key = key)
print(gndenergy[0][0])

0.0002198966121783299


##### Entropy

In [298]:
entropy1 = driver.get_bipartite_entanglement(kets[0])

###### HDF5 Files

In [299]:
entropy = pd.DataFrame(entropy1)
filename = generate_file_name_hdf5(params, entropy = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, entropy = True)
append_data_to_hdf5(params, entropy, filename, key = 'entropy', entropy = True)

##### Correlation Matrix

In [300]:
hop_exp_val = np.zeros((params['L'], params['L']), dtype=complex)
for i in range(params['L']):
    for j in range(i, params['L']):
        b = driver.expr_builder()
        b.add_term("CD", np.array([i, j]), 1)  # Hopping operator between i and j            
        hop_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
        hop_exp_val[i, j] = driver.expectation(kets[0], hop_mpo, kets[0])
        hop_exp_val[j, i] = np.conjugate(hop_exp_val[i, j])

In [301]:
correl_matrix = pd.DataFrame(hop_exp_val)
filename = generate_file_name_hdf5(params, Correlation = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, Correlation = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

##### NumberOperator and Its Standard Deviation

In [302]:
std_val, std_nOp = 0, 0
_exp_nop = []
for i in range(params['L']):
    b = driver.expr_builder()
    b.add_term("N", np.array([i]), 1)
## Calculate <N>   
    nOp_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    impo = driver.get_identity_mpo()

    nOp_exp = driver.expectation(kets[0], nOp_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0])
    _exp_nop.append(nOp_exp)
## Calculate <N^2>
    b = driver.expr_builder()
    b.add_term("NN", np.repeat([i], 2), 1)
    
    NN_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    NN_exp = driver.expectation(kets[0], NN_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0]) 
## Calculate Standard Deviation
    std_val+=np.sqrt(NN_exp - (nOp_exp*nOp_exp))        
std_nOp = (std_val/params['L'])

###### Expectation value of Number Operator on each site

In [303]:
density_nop = pd.DataFrame(_exp_nop)
filename = generate_file_name_hdf5(params, expnop = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, expnop = True)
key = generate_unique_key(params, exp_nOp = True)
append_data_to_hdf5(params, density_nop, filename, key = key, expnop = True)

In [304]:
# key = generate_unique_key(params, exp_nOp = True)
# density_nop = load_matrix_from_hdf5(filename1, key = key)
# print(density_nop)

###### Standard Deviation

In [305]:
std_deviation_numberOp = pd.DataFrame({'std_nOp':[std_nOp]})
filename = generate_file_name_hdf5(params)
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, std_deviation_numberOp = True)

## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, std_deviation_numberOp, filename1, key = key)

####### Use this only for the first time when taking data for a theta #######
## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, std_deviation_numberOp, filename, key = key)

In [306]:
key = generate_unique_key(params, std_deviation_numberOp = True)
std_nop = load_matrix_from_hdf5(filename1, key = key)
print(std_nop[0][0])

(0.5527393939814338+1.7765854273533978e-17j)


In [307]:
with pd.HDFStore(filename1, 'r') as hdf:
    # List all keys in the file
    keys = hdf.keys()
    print("Keys in HDF5 file:")
    for key in keys:
        print(key)

Keys in HDF5 file:
/gndenergy_U1_9e0_t1_0e0
/std_deviation_numberOp_U1_9e0_t1_0e0


#### $U = 2.0$

In [308]:
start_time = time.time()
#########################################

params['u'] = 2
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4 + [200] * 4 + [250] * 4 + [300] * 4 + [350] * 4 + [400] * 4 + [450] * 4 + [500] * 4 + [550] * 4 + [600] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)

#########################################
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

Execution time: 147.43 seconds


In [309]:
kets = [driver.split_mps(mps, ir, tag="KET-%d" % ir) for ir in range(mps.nroots)]

##### First Excitation Gap

In [310]:
gndenergy = pd.DataFrame({'gndenergy':[np.power(np.abs(energy[1]-energy[0]),2)]})
filename = generate_file_name_hdf5(params)
## Complete path
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, gndstate_energy = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, gndenergy, filename1, key = key)

In [311]:
## Testing
gndenergy = load_matrix_from_hdf5(filename1, key = key)
print(gndenergy[0][0])

0.00012875350232110868


##### Entropy

In [312]:
entropy1 = driver.get_bipartite_entanglement(kets[0])

###### HDF5 Files

In [313]:
entropy = pd.DataFrame(entropy1)
filename = generate_file_name_hdf5(params, entropy = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, entropy = True)
append_data_to_hdf5(params, entropy, filename, key = 'entropy', entropy = True)

##### Correlation Matrix

In [314]:
hop_exp_val = np.zeros((params['L'], params['L']), dtype=complex)
for i in range(params['L']):
    for j in range(i, params['L']):
        b = driver.expr_builder()
        b.add_term("CD", np.array([i, j]), 1)  # Hopping operator between i and j            
        hop_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
        hop_exp_val[i, j] = driver.expectation(kets[0], hop_mpo, kets[0])
        hop_exp_val[j, i] = np.conjugate(hop_exp_val[i, j])

In [315]:
correl_matrix = pd.DataFrame(hop_exp_val)
filename = generate_file_name_hdf5(params, Correlation = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, Correlation = True)
## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, correl_matrix, filename1, key = 'correl_matrix', Correlation = True)

##### NumberOperator and Its Standard Deviation

In [316]:
std_val, std_nOp = 0, 0
_exp_nop = []
for i in range(params['L']):
    b = driver.expr_builder()
    b.add_term("N", np.array([i]), 1)
## Calculate <N>   
    nOp_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    impo = driver.get_identity_mpo()

    nOp_exp = driver.expectation(kets[0], nOp_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0])
    _exp_nop.append(nOp_exp)
## Calculate <N^2>
    b = driver.expr_builder()
    b.add_term("NN", np.repeat([i], 2), 1)
    
    NN_mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
    NN_exp = driver.expectation(kets[0], NN_mpo, kets[0]) / driver.expectation(kets[0], impo, kets[0]) 
## Calculate Standard Deviation
    std_val+=np.sqrt(NN_exp - (nOp_exp*nOp_exp))        
std_nOp = (std_val/params['L'])

###### Expectation value of Number Operator on each site

In [317]:
density_nop = pd.DataFrame(_exp_nop)
filename = generate_file_name_hdf5(params, expnop = True)
## Complete path of the location of the file
filename1 = generating_complete_path_hdf5(filename, expnop = True)
key = generate_unique_key(params, exp_nOp = True)
append_data_to_hdf5(params, density_nop, filename, key = key, expnop = True)

In [318]:
# key = generate_unique_key(params, exp_nOp = True)
# density_nop = load_matrix_from_hdf5(filename1, key = key)
# print(density_nop)

###### Standard Deviation

In [319]:
std_deviation_numberOp = pd.DataFrame({'std_nOp':[std_nOp]})
filename = generate_file_name_hdf5(params)
filename1 = generating_complete_path_hdf5(filename)
key = generate_unique_key(params, std_deviation_numberOp = True)

## Use the code below to append data to an existing file/folder directory
append_data_to_hdf5(params, std_deviation_numberOp, filename1, key = key)

####### Use this only for the first time when taking data for a theta #######
## Use this code to write data to a new file irrespective of the presence of subholder based on 'L'.
# create_folders_and_store_data(params, std_deviation_numberOp, filename, key = key)

In [320]:
key = generate_unique_key(params, std_deviation_numberOp = True)
std_nop = load_matrix_from_hdf5(filename1, key = key)
print(std_nop[0][0])

(0.536188733530277-1.2909286344418562e-18j)


In [321]:
with pd.HDFStore(filename1, 'r') as hdf:
    # List all keys in the file
    keys = hdf.keys()
    print("Keys in HDF5 file:")
    for key in keys:
        print(key)

Keys in HDF5 file:
/gndenergy_U2_0e0_t1_0e0
/std_deviation_numberOp_U2_0e0_t1_0e0
